# Imports básicos para todas as análises

In [1]:
import os, json, warnings, shap

import \
    numpy as np, \
    optuna as opt, \
    pandas as pd, \
    matplotlib.pyplot as plt, \
    plotly.graph_objects as go, \
    plotly.express as px

from datetime import datetime
from plotly.subplots import make_subplots

from scipy.stats import norm

from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, FunctionTransformer, OrdinalEncoder
from sklearn.compose import ColumnTransformer

from statsforecast import StatsForecast
from statsforecast.models import SeasonalNaive

from sklearn.linear_model import LinearRegression
from catboost import CatBoostRegressor
from sklearn.ensemble import RandomForestRegressor

from skforecast.model_selection import backtesting_forecaster
from skforecast.ForecasterAutoreg import ForecasterAutoreg

# A ser usado apenas para a análise de imputação de dados (ao invés de sempre aplicar o valor médio)
from sklearn.impute import KNNImputer

# from sktime.param_est.seasonality import SeasonalityACF
# from sktime.param_est.stationarity import StationarityADF
from sktime.performance_metrics.forecasting import (
    MeanAbsolutePercentageError,
    MeanSquaredError,
)

# from sktime.split import temporal_train_test_split
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import (
    acf,
    pacf
)

# Desativar as mensagens de 'warning' que ficam poluindo o output de alguns trechos de código.
warnings.filterwarnings("ignore")

# Para com a verborragia do log do Optuna
opt.logging.set_verbosity(opt.logging.WARNING)

# Wraper pra usar a engine do Plotly ao invocar a função "[DataFrame|Series].plot" do Pandas
# pd.options.plotting.backend = "plotly"

# Métricas utilizadas
mape = MeanAbsolutePercentageError(symmetric=False) # Melhor valor possível é 0.0
rmse = MeanSquaredError(square_root=True) # Quanto menor, melhor

SEED         = 1984
NJOBS        = os.cpu_count()//2
NFOLDS       = 15
SALVAR_PLOTS = False

pasta_resultados = "./resultados/trecho_baixo/"

# Dicionário com as datas de início e término das estações
estacoes = {
    'verao'     : [(pd.Timestamp('12-21'), pd.Timestamp('03-20'))],
    'outono'    : [(pd.Timestamp('03-21'), pd.Timestamp('06-20'))],
    'inverno'   : [(pd.Timestamp('06-21'), pd.Timestamp('09-22'))],
    'primavera' : [(pd.Timestamp('09-23'), pd.Timestamp('12-20'))]
}

# Mapeamento das estações para números
estacao_para_numero = {
    'verao'     : 1,
    'outono'    : 2,
    'inverno'   : 3,
    'primavera' : 4
}

shap.initjs()

# Utilidades

### Métricas comumente aplicadas à Hidrologia

In [2]:
def kge(y_true, y_pred):
    """
        Calcula a métrica Kling-Gupta Efficiency (KGE), versão de 2009.
        Maior é melhor (Ótimo = 1)
        Limites=(-inf, 1]

        Parâmetros:
        y_true (array-like): Valores observados.
        y_pred (array-like): Valores previstos.

        Retorna:
        float: Valor da métrica KGE.
    """
    if not isinstance(y_true, np.ndarray):
        y_true = np.array(y_true)

    if not isinstance(y_pred, np.ndarray):
        y_pred = np.array(y_pred)  
    
    # Correlação linear
    r = np.corrcoef(y_true, y_pred)[0, 1]
    
    # Razão dos desvios padrão (beta)
    std_true = np.std(y_true, ddof=1)
    std_pred = np.std(y_pred, ddof=1)
    beta = std_pred / std_true
    
    # Razão das médias (gamma)
    mean_true = np.mean(y_true)
    mean_pred = np.mean(y_pred)
    gamma = mean_pred / mean_true
    
    # Cálculo do KGE
    result = 1 - np.sqrt(
        ((r - 1) ** 2) +
        ((beta - 1) ** 2) +
        ((gamma - 1) ** 2)
    )
    
    return result
# ##############################################################################################
def nse(y_true, y_pred):
    """
        Calcula a métrica Nash-Sutcliffe Efficiency (NSE).
        Maior é melhor (Ótimo = 1)
        Limites = (-inf, 1]

        Parâmetros:
        y_true (array-like): Valores observados.
        y_pred (array-like): Valores previstos.

        Retorna:
        float: Valor da métrica NSE.
    """
    if not isinstance(y_true, np.ndarray):
        y_true = np.array(y_true)

    if not isinstance(y_pred, np.ndarray):
        y_pred = np.array(y_pred)  
    
    # Média dos valores observados
    mean_y_true = np.mean(y_true)
    
    # Soma dos quadrados dos resíduos
    ss_res = np.sum((y_true - y_pred) ** 2)
    
    # Soma dos quadrados totais
    ss_tot = np.sum((y_true - mean_y_true) ** 2)
    
    # Cálculo do NSE
    result = 1 - (ss_res / ss_tot)
    
    return result
# ##############################################################################################
def pbias(y_true, y_pred):
    """
        O Viés Percentual - Percentual Bias (PBIAS) mede a tendência média dos valores simulados de serem maiores ou menores que os observados.
        O valor ideal de PBIAS é 0.0, com valores de baixa magnitude indicando simulação precisa do modelo.
        Valores positivos indicam viés de *SUPERESTIMAÇÃO*
        Valores negativos indicam viés de *SUBESTIMAÇÃO*

        Fonte: <https://www.mdpi.com/2073-4441/15/9/1693>

        Parâmetros:
        y_true (array-like): Valores observados.
        y_pred (array-like): Valores previstos.

        Retorna:
        float: Valor do PBIAS em porcentagem.
    """
    if not isinstance(y_true, np.ndarray):
        y_true = np.array(y_true)

    if not isinstance(y_pred, np.ndarray):
        y_pred = np.array(y_pred)  
  
    result = (np.sum(y_pred - y_true) / np.sum(y_true)) * 100
    
    return result
# ##############################################################################################
# def deviation_runoff_volume(y_true, y_pred):
#     """
#         Calcula o Deviation of the Runoff Volumes
#         Valor próximo de 1.0 indica que o modelo está indo bem nas previsões

#         Parâmetros:
#         y_true (array-like): Valores observados.
#         y_pred (array-like): Valores previstos.

#         Retorna:
#         float: Valor do DRV.
#     """
#     if not isinstance(y_true, np.ndarray):
#         y_true = np.array(y_true)

#     if not isinstance(y_pred, np.ndarray):
#         y_pred = np.array(y_pred)  
  
#     drv = np.sum(y_pred) / np.sum(y_true)
    
#     return drv
# ##############################################################################################
# def kling_gupta_efficiency_non_parametric(y_true, y_pred):
#     """
#         Código retirado de: <https://github.com/ThibHlln/hydroeval/blob/main/hydroeval/objective_functions.py>

#         Calcula a métrica Kling-Gupta Efficiency não-paramétrica (KGEnp).
#         Maior é melhor (Ótimo = 1)
#         Limites=(-inf, 1]

#         Traditional Kling-Gupta efficiencies (Gupta et al., 2009; Kling et al., 2012) range from -Inf to 1, and therefore KGEnp should do so.
#         Essentially, the closer to 1, the more similar 'y_pred' and 'y_true' are.
#         Knoben et al. (2019) showed that traditional Kling-Gupta (Gupta et al., 2009; Kling et al., 2012)
#             values greater than -0.41 indicate that a model improves upon the mean flow benchmark, even if the model's KGE value is negative.

#             Texto retirado de: <https://rdrr.io/cran/hydroGOF/man/KGEnp.html>

#     """

#     if not isinstance(y_true, np.ndarray):
#         y_true = np.array(y_true)

#     if not isinstance(y_pred, np.ndarray):
#         y_pred = np.array(y_pred) 

#     # calculate error in timing and dynamics r
#     # (Spearman's correlation coefficient)
#     sim_rank = np.argsort(np.argsort(y_pred, axis=0), axis=0)
#     obs_rank = np.argsort(np.argsort(y_true, axis=0), axis=0)

#     r_num = np.sum(
#         (obs_rank - np.mean(obs_rank, axis=0, dtype=np.float64)) *
#         (sim_rank - np.mean(sim_rank, axis=0, dtype=np.float64)),
#         axis=0
#     )

#     r_den = np.sqrt(
#         np.sum(
#             (obs_rank - np.mean(obs_rank, axis=0, dtype=np.float64)) ** 2,
#             axis=0
#         ) *
#         np.sum(
#             (sim_rank - np.mean(sim_rank, axis=0, dtype=np.float64)) ** 2,
#             axis=0
#         )
#     )

#     r = r_num / r_den

#     # calculate error in timing and dynamics alpha (flow duration curve)
#     sim_fdc = np.sort(
#         y_pred / (y_pred.shape[0] * np.mean(y_pred, axis=0, dtype=np.float64)),
#         axis=0
#     )

#     obs_fdc = np.sort(
#         y_true / (y_true.shape[0] * np.mean(y_true, axis=0, dtype=np.float64)),
#         axis=0
#     )

#     alpha = 1 - 0.5 * np.sum(np.abs(sim_fdc - obs_fdc), axis=0)

#     # calculate error in volume beta (bias of mean discharge)
#     beta = (np.mean(y_pred, axis=0) / np.mean(y_true, axis=0, dtype=np.float64))

#     # calculate the non-parametric Kling-Gupta Efficiency KGEnp
#     kgenp = 1 - np.sqrt(
#         ((r - 1) ** 2) +
#         ((alpha - 1) ** 2) +
#         ((beta - 1) ** 2)
#     )

#     return kgenp
# ##############################################################################################
# def r2(y_true, y_pred):
#     """
#         Calcula o Coeficiente de Determinação (R²).
#         Maior é melhor (Ótimo = 1)
#         Limites=[0, 1]

#         Parâmetros:
#         y_true (array-like): Valores observados.
#         y_pred (array-like): Valores previstos.

#         Retorna:
#         float: Valor do R².
#     """
#     if not isinstance(y_true, np.ndarray):
#         y_true = np.array(y_true)

#     if not isinstance(y_pred, np.ndarray):
#         y_pred = np.array(y_pred)  
    
#     # Média dos valores observados
#     mean_y_true = np.mean(y_true)
    
#     # Soma dos quadrados dos resíduos
#     ss_res = np.sum((y_true - y_pred) ** 2)
    
#     # Soma dos quadrados totais
#     ss_tot = np.sum((y_true - mean_y_true) ** 2)
    
#     # Cálculo do R²
#     result = 1 - (ss_res / ss_tot)
    
#     return result

### Funções utilitárias de plot, entre outras

In [3]:
def plot_serie_temporal(
    dataset          : pd.DataFrame,
    coluna           : str,
    tp_coluna        : str,
    plot_title       : str,
    line_color       : str,
    short_name       : str,
    pasta_resultados : str = "./",
    salvar           : bool = False,
) -> None:
    """
        Método para desenhar o gráfico da Série Temporal completa.

        Parâmetros:
            dataset: o DataFrame com os dados da série temporal para desenhar o gráfico

            coluna: a coluna para a qual se deseja gerar o gráfico

            tp_coluna: é um campo do tipo booleano que faz a distinção entre "vazão" e "chuva" (os dois únicos tipos de séries temporais que temos)
                True -> significa "vazão"
                False -> significa "chuva"

            plot_title: uma string com um título para o gráfico

            line_color: uma cor desejada para a linha desenhada da série temporal

            short_name: uma string que será usada para escrever a legenda do gráfico
            
            salvar: se deverá salvar o gráfico em disco (quando True) ou desenhar na tela (quando False)
    """

    fig = go.Figure()

    fig.add_trace(
        go.Scatter(
            x=dataset["ds"],
            y=dataset[coluna],
            mode="lines",
            name=short_name,
            line=dict(color=line_color, width=2)
        ),
    )

    fig.update_yaxes(
        zerolinecolor="black",
        mirror=True,
        ticks="outside",
        showline=True,
        linecolor="black",
        title=dict(text="Vazão (m³/s)" if tp_coluna == "vazao" else "Precipitação (mm/dia)", font=dict(family="system-ui", size=20))
    )

    fig.update_xaxes(
        mirror=True,
        ticks="outside",
        showline=True,
        linecolor="black",
        title=dict(text="Período", font=dict(family="system-ui", size=20))
    )

    fig.update_layout(
        width=1150,
        height=550,
        hovermode="x unified",
        plot_bgcolor="#c8d4e3",
        xaxis=dict(tickfont=dict(family="system-ui", size=16)),
        yaxis=dict(tickfont=dict(family="system-ui", size=16)),
        title=dict(text=plot_title, font=dict(family="system-ui", size=26)),
    )

    if salvar:
        now = datetime.now()
        fig.write_image(pasta_resultados + f"SerieTemporal_col[{coluna}]_{now.strftime('%Y-%m-%d_%H-%M-%S')}.png")
    else:
        fig.show()
# ============================================================================================ #
def decomp_series(
    df           : pd.DataFrame,
    observado    : bool,
    tendencia    : bool,
    sazonalidade : bool,
    residuo      : bool,
    cor_linha    : str,
    salvar       : bool = False,
) -> None:
    # A decomposição das séries temporais ajuda a detectar padrões (tendência, sazonalidade)
    #   e identificar outras informações que podem ajudar na interpretação do que está acontecendo.

    # cols = df.drop(columns=["ds", "unique_id"]).columns.to_list()
    cols = df.drop(columns=["ds"]).columns.to_list()
    for c in cols:
        
        # Utilizei modelo do tipo "add" (aditivo) pois tem séries com valores 0 (zero).
        # Período de 365 dias porque o que me interessa é capturar comportamentos anuais.
        decomp = seasonal_decompose(
            df[c],
            period=365, # 365 dias = 1 ano
            model="add"
        )

        fig_decomp = make_subplots(specs=[[{"secondary_y": True}]])

        if observado:
            fig_decomp.add_trace(
                go.Scatter(
                    x=df.ds,
                    y=decomp.observed,
                    name="observado",
                    mode="lines",
                    showlegend=True,
                    line=dict(color=cor_linha),
                ),
                secondary_y=False,
            )

        if tendencia:
            fig_decomp.add_trace(
                go.Scatter(
                    x=df.ds,
                    y=decomp.trend,
                    name="tendência",
                    mode="lines",
                    showlegend=True,
                    line=dict(color=cor_linha),
                ),
                secondary_y=True,
            )

        if sazonalidade:
            fig_decomp.add_trace(
                go.Scatter(
                    x=df.ds,
                    y=decomp.seasonal,
                    name="sazonalidade",
                    mode="lines",
                    showlegend=True,
                    line=dict(color=cor_linha),
                ),
                secondary_y=True,
            )

        if residuo:
            fig_decomp.add_trace(
                go.Scatter(
                    x=df.ds,
                    y=decomp.resid,
                    name="resíduo",
                    mode="lines",
                    showlegend=True,
                    line=dict(color=cor_linha),
                ),
                secondary_y=False,
            )

        fig_decomp.update_yaxes(
            # title=dict(
            #     text="observado/resíduo",
            #     font=dict(family="system-ui", size=20)
            # ),
            secondary_y=False,
            zerolinecolor="black",
            mirror=True,
            ticks="outside",
            showline=True,
            linecolor="black",
        )

        fig_decomp.update_yaxes(
            # title=dict(
            #     text="tendência/sazonalidade",
            #     font=dict(family="system-ui", size=20)
            # ),
            secondary_y=True,
            mirror=True,
            ticks="outside",
            showline=True,
            linecolor="black",
        )

        fig_decomp.update_xaxes(
            mirror=True,
            ticks="outside",
            showline=True,
            linecolor="black",
            title=dict(text="Período", font=dict(family="system-ui", size=20))
        )

        fig_decomp.update_layout(
            width=1150,
            height=550,
            plot_bgcolor="#c8d4e3",
            hovermode="x unified",
            xaxis=dict(tickfont=dict(family="system-ui", size=16)),
            yaxis=dict(tickfont=dict(family="system-ui", size=16)),
            title=dict(text=f"Decomposição da série temporal: {c}", font=dict(family="system-ui", size=26)),
        )

        if salvar:
            fig_decomp.write_image(pasta_resultados + f"aed/decomposicao_serie_{c}.png")
        else:
            fig_decomp.show()
# ============================================================================================ #
def mapa_correlacao(
    df       : pd.DataFrame,
    medida   : str = "pearson",
    autosize : bool = False,
    salvar   : bool = False
) -> None:

    if medida == "dtw":
        from dtaidistance import dtw

        dtw_dist = dtw.distance_matrix_fast(df.drop(columns=["ds"]).T.values)
        # dtw_dist = dtw.distance_matrix_fast(df.drop(columns=["ds", "unique_id"]).T.values)
        
        df_dtw_dist = pd.DataFrame(
            data=dtw_dist,
            # index=df.drop(columns=["ds", "unique_id"]).columns.to_list(),
            # columns=df.drop(columns=["ds", "unique_id"]).columns.to_list(),
            index=df.drop(columns=["ds"]).columns.to_list(),
            columns=df.drop(columns=["ds"]).columns.to_list(),
        )

        fig = go.Figure()

        fig.add_trace(
            go.Heatmap(
                x=df_dtw_dist.columns,
                y=df_dtw_dist.columns,
                z=df_dtw_dist,
                text=df_dtw_dist.values,
                texttemplate="%{text:.7f}",
                textfont={"size": 14},
                colorscale="rainbow",
                hovertemplate="%{y}<br>%{x}</br><extra></extra>",
            )
        )

        fig.update_yaxes(
            tickfont=dict(family="system-ui", size=16),
            mirror=True,
            ticks="outside",
            showline=True,
            linecolor="black",
        )

        fig.update_xaxes(
            tickfont=dict(family="system-ui", size=16),
            mirror=True,
            ticks="outside",
            showline=True,
            linecolor="black",
        )

        fig.update_layout(
            width=1150,
            height=550,
            title=dict(text="Mapa de correlação (DTW)", font=dict(family="system-ui", size=26)
            ),
        )

    elif medida == "pearson":

        # corr = df.drop(columns=["ds", "unique_id"]).corr()
        corr = df.drop(columns=["ds"]).corr()

        fig = go.Figure()

        fig.add_trace(
            go.Heatmap(
                x=corr.columns,
                y=corr.columns,
                z=corr,
                text=corr.values,
                texttemplate="%{text:.7f}",
                textfont={"size": 14},
                colorscale="rainbow",
                hovertemplate="%{y}<br>%{x}</br><extra></extra>",
            )
        )

        fig.update_yaxes(
            tickfont=dict(family="system-ui", size=16),
            mirror=True,
            ticks="outside",
            showline=True,
            linecolor="black",
        )

        fig.update_xaxes(
            tickfont=dict(family="system-ui", size=16),
            mirror=True,
            ticks="outside",
            showline=True,
            linecolor="black",
        )

        if autosize:
            fig.update_layout(
                autosize=True,
                title=dict(text="Mapa de correlação", font=dict(family="system-ui", size=26))
            )
        else:
            fig.update_layout(
            width=1150,
            height=550,
            title=dict(text="Mapa de correlação", font=dict(family="system-ui", size=26))
        )


    else:
        raise Exception("Opção errada. ('dtw' ou 'pearson')")

    if salvar:
        fig.write_image(pasta_resultados + f"aed/mapa_correlacao_{medida}.png")
    else:
        fig.show()
# ============================================================================================ #
def plot_linha_tabela(
    df_merged  : pd.DataFrame,
    regressor  : str,
    plot_title : str,
    line_color : str,
    short_name : str,
    salvar     : bool = False,
) -> None:

    fig = make_subplots(
        rows=2,
        cols=1,
        vertical_spacing=0.2,
        specs=[
            [{"type": "scatter"}],
            [{"type": "table"}]
        ],
    )

    fig.add_trace(
        go.Scatter(
            x=df_merged["ds"],
            y=df_merged["y"],
            mode="lines+markers",
            name="observado",
            line=dict(color="#000000", width=2)
        ),
        row=1,
        col=1,
    )

    fig.add_trace(
        go.Scatter(
            x=df_merged["ds"],
            y=df_merged[regressor],
            mode="lines+markers",
            name=short_name,
            line=dict(color=line_color),
        ),
        row=1,
        col=1,
    )

    fig.add_trace(
        go.Table(
            header=dict(
                values=[
                    "MAPE",
                    "RMSE",
                    "PBIAS (%)",
                ],
                font=dict(size=18),
                align="center"
            ),
            cells=dict(
                values=[
                    mape(y_true=df_merged.y.values, y_pred=df_merged[regressor].values),
                    rmse(y_true=df_merged.y.values, y_pred=df_merged[regressor].values),
                    pbias(y_true=df_merged.y.values, y_pred=df_merged[regressor].values),
                ],
                font=dict(size=18),
                height=30,
                align="left",
            ),
        ),
        row=2,
        col=1,
    )

    fig.update_yaxes(
        zerolinecolor="black",
        mirror=True,
        ticks="outside",
        showline=True,
        linecolor="black",
        title=dict(text="Vazão (m³/s)", font=dict(family="system-ui", size=20))
    )

    fig.update_xaxes(
        mirror=True,
        ticks="outside",
        showline=True,
        linecolor="black",
        title=dict(text="Período", font=dict(family="system-ui", size=20))
    )

    fig.update_layout(
        width=1150,
        height=700,
        hovermode="x unified",
        plot_bgcolor="#c8d4e3",
        xaxis=dict(tickfont=dict(family="system-ui", size=16)),
        yaxis=dict(tickfont=dict(family="system-ui", size=16)),
        title=dict(text=plot_title, font=dict(family="system-ui", size=26)),
    )

    if salvar:
        now = datetime.now()
        fig.write_image(pasta_resultados + f"{regressor}_{now.strftime('%Y-%m-%d_%H-%M-%S')}.png")
    else:
        fig.show()
# ============================================================================================ #
def cria_plot_correlacao(
    serie  : pd.Series,
    n_lags : int,
    coluna : str,
    titulo : str,
    subtitulo : str,
    salvar    : bool = False,
    plot_pacf : bool = False,
    linha_treino_teste : bool = True,
) -> None:

    corr_array = (
        pacf(serie.dropna(), nlags=n_lags, alpha=0.05)
        if plot_pacf
        else acf(serie.dropna(), nlags=n_lags, alpha=0.05)
    )

    lower_y = corr_array[1][:, 0] - corr_array[0]
    upper_y = corr_array[1][:, 1] - corr_array[0]

    fig = go.Figure()

    # Desenha as linhas verticais pretas
    [
        fig.add_scatter(
            x=(x, x),
            y=(0, corr_array[0][x]),
            mode="lines",
            line_color="black",
            hovertemplate="<extra></extra>",
        )
        for x in range(len(corr_array[0]))
    ]

    # Desenha as bolinhas vermelhas
    fig.add_scatter(
        x=np.arange(len(corr_array[0])),
        y=corr_array[0],
        mode="markers",
        marker_color="red",
        marker_size=12,
        hovertemplate="x = %{x}<br>y = %{y}<extra></extra>",
    )

    # Desenha a 'nuvem' clarinha acima do eixo x
    fig.add_scatter(
        x=np.arange(len(corr_array[0])),
        y=upper_y,
        mode="lines",
        line_color="rgba(255,255,255,0)",
        hovertemplate="<extra></extra>",
    )

    # Desenha a 'nuvem' clarinha abaixo do eixo x
    fig.add_scatter(
        x=np.arange(len(corr_array[0])),
        y=lower_y,
        mode="lines",
        fillcolor="rgba(32, 146, 230,0.3)",
        fill="tonexty",
        line_color="rgba(255,255,255,0)",
        hovertemplate="<extra></extra>",
    )

    fig.update_traces(showlegend=False)

    fig.update_xaxes(
        range=[-1, n_lags + 1],
        mirror=True,
        ticks="outside",
        showline=True,
        linecolor="black",
        title=dict(text=coluna, font=dict(family="system-ui", size=20)),
    )

    fig.update_yaxes(
        zerolinecolor="black",  # Quando 'y=0' a linha é preta
        mirror=True,
        ticks="outside",
        showline=True,
        linecolor="black",
    )

    # Adicionar linha vertical
    if linha_treino_teste:
        fig.add_shape(
            type="line",
            x0=365, y0=0,
            x1=365, y1=1,
            line=dict(color="black", width=2, dash="solid"),
            name='Lag 365'
        )

    fig.update_layout(
        width=1150,
        height=550,
        plot_bgcolor="#c8d4e3",
        xaxis=dict(tickfont=dict(family="system-ui", size=16)),
        yaxis=dict(tickfont=dict(family="system-ui", size=16)),
        title=dict(text=f"{titulo}<br><sup>{subtitulo}</sup>", font=dict(family="system-ui", size=26)),
    )

    if salvar:
        (
            fig.write_image(pasta_resultados + "aed/plot_pacf.png")
            if plot_pacf
            else fig.write_image(pasta_resultados + "aed/plot_acf.png")
        )
    else:
        fig.show()
# ============================================================================================ #
def exportar_dict_json(
    v_dict   : dict,
    pasta    : str,
    nome_arq : str
) -> None:
    
    if not os.path.exists(pasta):
        os.makedirs(pasta)

    json_str = json.dumps(v_dict, indent=4)
    with open(pasta + nome_arq, "w") as a:
        a.write(json_str)
# ============================================================================================ #
def plot_divisao_treino_teste(
    df_treino : pd.DataFrame,
    df_teste  : pd.DataFrame,
    index_ds  : bool,
    col_nome  : str,
    col_data  : str = "ds",
    col_plot  : str = "y",
    salvar    : bool = False,
) -> None:
    
    df_original = pd.concat([df_treino, df_teste])

    fig = go.Figure()

    fig.add_trace(
        go.Scatter(
            x=df_treino.index if index_ds else df_treino[col_data],
            y=df_treino[col_plot],
            mode="lines",
            name="treino"
        )
    )

    fig.add_trace(
        go.Scatter(
            x=df_teste.index if index_ds else df_teste[col_data],
            y=df_teste[col_plot],
            mode="lines",
            name="teste"
        )
    )

    fig.update_yaxes(
        zerolinecolor="black",
        mirror=True,
        ticks="outside",
        showline=True,
        linecolor="black",
        title=dict(text="Vazão (m³/s)", font=dict(family="system-ui", size=18))
    )

    fig.update_xaxes(
        mirror=True,
        ticks="outside",
        showline=True,
        linecolor="black",
        title=dict(text="Período", font=dict(family="system-ui", size=18))
    )

    # Adicionar linha vertical para separação treino/teste
    fig.add_shape(
        type="line",
        x0=max(df_treino.index), y0=0,
        x1=max(df_treino.index), y1=(max(df_original.y)+200),
        line=dict(color="black", width=2, dash="solid"),
        name='Separação Treino/Teste'
    )
    
    fig.update_layout(
        width=1150,
        height=550,
        hovermode="x unified",
        plot_bgcolor="#c8d4e3",
        xaxis=dict(tickfont=dict(family="system-ui", size=16)),
        yaxis=dict(tickfont=dict(family="system-ui", size=16)),
        title=dict(text=f"Vazão 'y' ({col_nome})", font=dict(family="system-ui", size=26)),
    )

    if salvar:
        fig.write_image(pasta_resultados + f"aed/divisao_treino_teste_{col_nome}.png")
    else:
        fig.show()
# ============================================================================================ #
def plot_resultados(
    df_merged  : pd.DataFrame,
    modelo     : str,
    nome_curto : str,
    fh         : int,
    titulo     : str,
    subtitulo  : str,
    pasta_dstn : str,
    niveis     : list = None,
    cores      : list = None,
    salvar     : bool = False,
    n_decimal  : int = 5,
    metricas   : str = "padrao",
    marcadores : bool = True,
    indice_ds  : bool = False,
) -> None:
    """
        df_merged:
            O DataFrame com todas as previsões, os valores observados e os quartis (se houver)

        modelo:
            String com o nome do modelo que será referenciado no DataFrame (df_merged[modelo])

        nome_curto:
            Nome curto do modelo para colocar na legenda do gráfico

        fh:
            Horizonte de previsão para colocar no título do gráfico

        titulo:
            O título do gráfico

        pasta_dstn:
            Pasta onde salvar a imagem

        niveis:
            Array, em ordem do maior para o menor, com os níveis dos quartis

        cores:
            Cores para aplicar ao plotar os quartis no gráfico

        salvar:
            Se vai salvar diretamente para um arquivo ou se vai renderizar em tela o gráfico

        n_decimal:
            Para arredondar as casas decimais dos números na tabela com as métricas

        metricas:
            Se vai gerar a tabela com as métricas resumidas com as métricas padrão ou as mais comumente usadas em Hidrologia

        marcadores:
            Se as linhas dos gráficos serão lisas ou com marcadores
    """

    mtrcs = {}
    if metricas == "padrao":
        mtrcs[modelo] = {
            "MAPE": mape(y_true=df_merged["y"].values, y_pred=df_merged[modelo].values),
            "RMSE": rmse(y_true=df_merged["y"].values, y_pred=df_merged[modelo].values),
            "PBIAS (%)": pbias(y_true=df_merged["y"].values, y_pred=df_merged[modelo].values),
        }

        if niveis is not None:
            for n in niveis:
                mtrcs[modelo+"-lo-"+n] = {
                    "MAPE": mape(y_true=df_merged["y"].values, y_pred=df_merged[modelo+"-lo-"+n].values),
                    "RMSE": rmse(y_true=df_merged["y"].values, y_pred=df_merged[modelo+"-lo-"+n].values),
                    "PBIAS (%)": pbias(y_true=df_merged["y"].values, y_pred=df_merged[modelo+"-lo-"+n].values),
                }

                mtrcs[modelo+"-hi-"+n] = {
                    "MAPE": mape(y_true=df_merged["y"].values, y_pred=df_merged[modelo+"-hi-"+n].values),
                    "RMSE": rmse(y_true=df_merged["y"].values, y_pred=df_merged[modelo+"-hi-"+n].values),
                    "PBIAS (%)": pbias(y_true=df_merged["y"].values, y_pred=df_merged[modelo+"-hi-"+n].values),
                }
    elif metricas == "hidrologia":
        mtrcs[modelo] = {
            "MAPE": mape(y_true=df_merged["y"].values, y_pred=df_merged[modelo].values),
            "KGE": kge(y_true=df_merged["y"].values, y_pred=df_merged[modelo].values),
            "PBIAS (%)": pbias(y_true=df_merged["y"].values, y_pred=df_merged[modelo].values),
        }

        if niveis is not None:
            for n in niveis:
                mtrcs[modelo+"-lo-"+n] = {
                    "MAPE": mape(y_true=df_merged["y"].values, y_pred=df_merged[modelo+"-lo-"+n].values),
                    "KGE": kge(y_true=df_merged["y"].values, y_pred=df_merged[modelo+"-lo-"+n].values),
                    "PBIAS (%)": pbias(y_true=df_merged["y"].values, y_pred=df_merged[modelo+"-lo-"+n].values),
                }

                mtrcs[modelo+"-hi-"+n] = {
                    "MAPE": mape(y_true=df_merged["y"].values, y_pred=df_merged[modelo+"-hi-"+n].values),
                    "KGE": kge(y_true=df_merged["y"].values, y_pred=df_merged[modelo+"-hi-"+n].values),
                    "PBIAS (%)": pbias(y_true=df_merged["y"].values, y_pred=df_merged[modelo+"-hi-"+n].values),
                }
    else:
        raise Exception("Opção inválida para métrica! ('padrao' | 'hidrologia')")

    df_tbl = pd.DataFrame(mtrcs).T.reset_index(names="Modelo").round(n_decimal) # Arredondando para "n_decimal" casas decimais

    fig = make_subplots(
        rows=3,
        cols=1,
        vertical_spacing=0.1,
        specs=[
            [{"type": "scatter"}],
            [{"type": "table"}],
            [{"type": "table"}]
        ],
    )

    if niveis is not None and cores is not None:
        for n, c in zip(niveis, cores):
            fig.add_trace(
                go.Scatter(
                    x=df_merged.index if indice_ds else df_merged["ds"],
                    y=df_merged[modelo+"-hi-"+n],
                    mode="lines+markers" if marcadores else "lines",
                    name=nome_curto+"-hi-"+n,
                    line=dict(color=c),
                ),
                row=1,
                col=1,
            )

            fig.add_trace(
                go.Scatter(
                    x=df_merged.index if indice_ds else df_merged["ds"],
                    y=df_merged[modelo+"-lo-"+n],
                    mode="lines+markers" if marcadores else "lines",
                    name=nome_curto+"-lo-"+n,
                    fill="tonexty",
                    line=dict(color=c),
                ),
                row=1,
                col=1,
            )

    fig.add_trace(
        go.Scatter(
            x=df_merged.index if indice_ds else df_merged["ds"],
            y=df_merged["y"],
            mode="lines+markers" if marcadores else "lines",
            name="observado",
            line=dict(color="black", width=2),
        ),
        row=1,
        col=1,
    )
    
    fig.add_trace(
        go.Scatter(
            x=df_merged.index if indice_ds else df_merged["ds"],
            y=df_merged[modelo],
            mode="lines+markers" if marcadores else "lines",
            name=nome_curto,
            line=dict(color="magenta", width=4),
        ),
        row=1,
        col=1,
    )

    fig.add_trace(
        go.Table(
            header=dict(values=df_tbl.columns.to_list(), font=dict(size=18), align="center"),
            cells=dict(values=df_tbl.T, font=dict(size=18), height=30, align="left"),
        ),
        row=2,
        col=1,
    )

    fig.add_trace(
        go.Table(
            header=dict(
                values=["Cobertura empírica do intervalo de previsão"],
                font=dict(size=18),
                align="center"
            ),
            cells=dict(
                values=[cobertura_empirica(
                    y=df_merged["y"],
                    lower_bound=df_merged[modelo+"-lo-"+n],
                    upper_bound=df_merged[modelo+"-hi-"+n]
                )],
                font=dict(size=18),
                height=30,
                align="left"
            ),
        ),
        row=3,
        col=1,
    )

    fig.update_yaxes(
        mirror=True,
        ticks="outside",
        showline=True,
        linecolor="black",
        title=dict(text="Vazão (m³/s)", font=dict(family="system-ui", size=20))
    )

    fig.update_xaxes(
        mirror=True,
        ticks="outside",
        showline=True,
        linecolor="black",
        # title=dict(text="Período", font=dict(family="system-ui", size=20))
        title=dict(text="", font=dict(family="system-ui", size=20))
    )

    fig.update_traces(
        hovertemplate=None,
        row=1,
        col=1
    )

    fig.update_layout(
        width=1150,
        height=750,
        plot_bgcolor="#c8d4e3",
        hovermode="x unified",
        xaxis=dict(tickfont=dict(family="system-ui", size=16)),
        yaxis=dict(tickfont=dict(family="system-ui", size=16)),
        title=dict(text=f"{titulo}<br><sup>{subtitulo}</sup>", font=dict(family="system-ui", size=26))
    )
  
    if salvar:
        if not os.path.exists(pasta_dstn):
            os.makedirs(pasta_dstn)

        now = datetime.now()
        fig.write_image(pasta_dstn + f"/{modelo}_fh{fh}_{now.strftime('%Y-%m-%d_%H-%M-%S')}.png")
    else:
        fig.show()
# ============================================================================================ #
def determinar_estacao(data, estacao):
    # Substituir 29 de fevereiro por 28 de fevereiro
    if data.month == 2 and data.day == 29:
        data = data.replace(day=28)

    data = pd.Timestamp(data.strftime('%m-%d'))  # Considerar apenas o mês e o dia
    for estacao, periodos in estacoes.items():
        for inicio, fim in periodos:
            if inicio <= data <= fim or (inicio > fim and (data >= inicio or data <= fim)):
                return estacao_para_numero[estacao]
    return 'desconhecida'
# ============================================================================================ #
# Função principal para adicionar colunas ao DataFrame
def adicionar_estacao(
        df          : pd.DataFrame,
        coluna_data : str
    ) -> pd.DataFrame:
    
    df['estacao'] = df[coluna_data].apply(lambda x: determinar_estacao(x, estacoes))
    return df
# ============================================================================================ #
def gerar_atributos_data(
    df        : pd.DataFrame,
    atributos : list[str],
    col_data  : str = "ds",
) -> pd.DataFrame:

  df_result = df.copy()
  if atributos is not None:
    for a in atributos:
      if a in ("week", "weekofyear"):
        w = pd.to_datetime(df_result[col_data]).dt.isocalendar()
        df_result[a] = getattr(w, a)
      elif a == "estacao":
        df_result = adicionar_estacao(df_result, col_data)
      else:
        df_result[a] = getattr(pd.to_datetime(df_result[col_data]).dt, a)

  return df_result
# ============================================================================================ #
# Preencher com dados do ano anterior (max_years=1) ou média dos anos anteriores (max_years>1)
def fill_with_previous_years_average(
        df        : pd.DataFrame,
        columns   : list[str],
        date_col  : str = 'ds',
        max_years : int = 3
    ) -> pd.DataFrame :

    df_filled = df.copy()
    for col in columns:
        for i in range(len(df)):
            if pd.isnull(df_filled.loc[i, col]):
                sum_values   = 0
                count_values = 0
                for year in range(1, max_years + 1):
                    previous_year_date  = df_filled.loc[i, date_col] - pd.DateOffset(years=year)
                    previous_year_value = df_filled.loc[df_filled[date_col] == previous_year_date, col]
                    if not previous_year_value.empty:
                        sum_values += previous_year_value.values[0]
                        count_values += 1
                if count_values > 0:
                    df_filled.loc[i, col] = sum_values / count_values
    return df_filled
# ============================================================================================ #
def plot_observado_previsao(df, modelo, col_y_pred, salvar, pasta_dstn):
    # Retirado de: <https://plotly.com/python/ml-regression/#simple-actual-vs-predicted-plot>
    fig = go.Figure()

    fig.add_trace(
        go.Scatter(
            x=df[col_y_pred],
            y=df["y"],
            mode="markers",
            line=dict(color="blue"),
            hovertemplate="previsão: %{x}<br>observado: %{y}</br><extra></extra>",
            showlegend=False
        )
    )

    fig.add_shape(
        type="line",
        line=dict(color='red', dash='dash'),
        x0=df["y"].min(), y0=df["y"].min(),
        x1=df["y"].max(), y1=df["y"].max()
    )

    fig.update_xaxes(
        zerolinecolor="black",
        showspikes=True,
        mirror=True,
        ticks="outside",
        showline=True,
        linecolor="black",
        title=dict(text="Previsão (m³/s)", font=dict(family="system-ui", size=20))
    )

    fig.update_yaxes(
        zerolinecolor="black",
        showspikes=True,
        mirror=True,
        ticks="outside",
        showline=True,
        linecolor="black",
        title=dict(text="Observado (m³/s)", font=dict(family="system-ui", size=18))
    )

    fig.update_layout(
        width=1150,
        height=550,
        hovermode="closest",
        plot_bgcolor="#c8d4e3",
        xaxis=dict(tickfont=dict(family="system-ui", size=16)),
        yaxis=dict(tickfont=dict(family="system-ui", size=16)),
        title=dict(text=f"Relação entre Observado x Previsto <br><sup>{modelo}</sup>", font=dict(family="system-ui", size=26))
    )

    if salvar:
        fig.write_image(pasta_dstn+"/relacao_observado_previsao.png")
    else:
        fig.show()
# ============================================================================================ #
def cobertura_empirica(y, lower_bound, upper_bound):
    """
        Calcula se o intervalo de previsão conseguiu cobrir o valor real observado
    """
    media = np.mean(
        np.logical_and(
            y >= lower_bound,
            y <= upper_bound
        )
    )
    return f"{round(100*media, 2)} %"
# ============================================================================================ #
def adicionar_colunas_agregadas(
        data_frame  : pd.DataFrame,
        nome_col    : str,
        x_dias      : int,
        tp_operacao : str
    ) -> pd.DataFrame:
    """
    Adiciona uma coluna ao DataFrame com a média ou soma dos últimos x dias.
    
    Parâmetros:
        - data_frame  : O DataFrame original.
        - nome_col    : Nome da coluna sobre a qual a operação será realizada.
        - x_dias      : Número de dias para calcular a média ou soma.
        - tp_operacao : Tipo de operação ('media' ou 'soma').
    
    Retorna:
        - DataFrame com a nova coluna adicionada.
    """
    
    # Verifica se a operação é válida
    if tp_operacao not in ['media', 'soma']:
        raise ValueError("O parâmetro 'tp_operacao' deve ser 'media' ou 'soma'.")

    # Calcula a nova coluna com base na operação solicitada
    if tp_operacao == 'media':
        data_frame[f'{nome_col}_media_{x_dias}_dias'] = data_frame[nome_col].rolling(window=x_dias).mean()
    elif tp_operacao == 'soma':
        data_frame[f'{nome_col}_soma_{x_dias}_dias'] = data_frame[nome_col].rolling(window=x_dias).sum()
    
    return data_frame
# ============================================================================================ #
def plot_residuos_previsoes(y_pred, residuos, titulo, subtitulo, cor, faixas):
    """
    Gráfico dos RESÍDUOS vs PREVISÕES
        Se não houver padrão visível (como um funil ou um cone), os resíduos apresentam variância constante (homoscedasticidade)
    """

    if faixas:
        q1 = np.percentile(residuos.values, 25)
        q3 = np.percentile(residuos.values, 75)
        iqr = q3 - q1

        lower_fence = q1 - (1.5 * iqr)
        upper_fence = q3 + (1.5 * iqr)

        x_range = [min(y_pred), max(y_pred)]
        fence_shaded_area = go.Scatter(
            x=x_range + x_range[::-1],
            y=[lower_fence, lower_fence, upper_fence, upper_fence],
            fill='toself',
            fillcolor='rgba(0, 100, 80, 0.2)',
            line=dict(color='rgba(255,255,255,0)'),
            name='Área ±1,5 * IQR'
        )

        lower_fence_line = go.Scatter(
            x=x_range,
            y=[lower_fence, lower_fence],
            mode="lines",
            line=dict(color="black", dash="dash"),
            name="Lower Fence"
        )

        upper_fence_line = go.Scatter(
            x=x_range,
            y=[upper_fence, upper_fence],
            mode="lines",
            line=dict(color="black", dash="dash"),
            name="Upper Fence"
        )

    scatter_plot = go.Scatter(
        x=y_pred,
        y=residuos,
        mode='markers',
        marker=dict(color=cor),
        name='Resíduos'
    )

    zero_line = go.Scatter(
        x=y_pred,
        y=[0]*len(y_pred),
        mode='lines',
        line=dict(color='red', dash='dash'),
        name='Linha de referência Y=0'
    )

    layout = go.Layout(
        width=1150,
        height=550,
        title=dict(
            text=f"{titulo}<br><sup>{subtitulo}</sup>",
            font=dict(family="system-ui", size=26)
        ),
        xaxis=dict(
            title     = "Valores Previstos",
            titlefont = dict(size=20),
            tickfont  = dict(size=16),
        ),
        yaxis=dict(
            title     = "Resíduos",
            titlefont = dict(size=20),
            tickfont  = dict(size=16),
        ),
        showlegend=True
    )

    if faixas:
        fig = go.Figure(data=[scatter_plot, fence_shaded_area, lower_fence_line, zero_line, upper_fence_line], layout=layout)
    else:
        fig = go.Figure(data=[scatter_plot, zero_line], layout=layout)
    fig.show()
# ============================================================================================ #
def plot_dist_residuos_periodo(residuos, titulo, subtitulo, cor, faixas):
    if faixas:
        q1 = np.percentile(residuos.values, 25)
        q3 = np.percentile(residuos.values, 75)
        iqr = q3 - q1

        lower_fence = q1 - (1.5 * iqr)
        upper_fence = q3 + (1.5 * iqr)

        x_range = [min(residuos.index), max(residuos.index)]
        fence_shaded_area = go.Scatter(
            x=x_range + x_range[::-1],
            y=[lower_fence, lower_fence, upper_fence, upper_fence],
            fill='toself',
            fillcolor='rgba(0, 100, 80, 0.2)',
            line=dict(color='rgba(255,255,255,0)'),
            name='Área ±1,5 * IQR'
        )

        lower_fence_line = go.Scatter(
            x=residuos.index,
            y=[lower_fence] * len(residuos),
            mode="lines",
            line=dict(color="black", dash="dash"),
            name="Lower Fence"
        )

        upper_fence_line = go.Scatter(
            x=residuos.index,
            y=[upper_fence] * len(residuos),
            mode="lines",
            line=dict(color="black", dash="dash"),
            name="Upper Fence"
        )

    disp_tempo = go.Scatter(
        x=residuos.index,
        y=residuos.values,
        mode='markers',
        line=dict(color=cor, width=2),
        name='Dispersão'
    )

    linha_referencia = go.Scatter(
        x=residuos.index,
        y=[0] * len(residuos),
        mode='lines',
        line=dict(color='red', width=2, dash='dash'),
        name='Linha de referência Y=0'
    )

    layout = go.Layout(
        width=1150,
        height=550,
        title=dict(
            text=f"{titulo}<br><sup>{subtitulo}</sup>",
            font=dict(family="system-ui", size=26)
        ),
        xaxis=dict(
            title     = "Período",
            titlefont = dict(size=20),
            tickfont  = dict(size=16),
        ),
        yaxis=dict(
            title     = "Resíduo",
            titlefont = dict(size=20),
            tickfont  = dict(size=16),
        ),
        showlegend=True
    )

    if faixas:
        fig = go.Figure(data=[disp_tempo, fence_shaded_area, linha_referencia, lower_fence_line, upper_fence_line], layout=layout)
    else:
        fig = go.Figure(data=[disp_tempo, linha_referencia], layout=layout)

    fig.show()
# ============================================================================================ #
def distribuicao_curva_normal(residuos, cor_histo, cor_linha, titulo, subtitulo):
    media_res = np.mean(residuos.values)
    std_res   = np.std(residuos.values)

    hist_data = go.Histogram(
        x=residuos,
        nbinsx=50,
        # histnorm ='probability density',  # Normaliza o histograma para densidade de probabilidade
        marker=dict(color=cor_histo),
        yaxis='y1',
        opacity=1.0,
        name='Histograma dos Resíduos'
    )

    xmin, xmax = residuos.min(), residuos.max()
    x = np.linspace(xmin, xmax, 100)
    y = norm.pdf(x, media_res, std_res)

    normal_curve = go.Scatter(
        x=x,
        y=y,
        mode='lines',
        line=dict(color=cor_linha, width=2),
        yaxis='y2',
        name='Curva Normal'
    )

    layout = go.Layout(
        width=1150,
        height=550,
        title=dict(
            # text=f"Distribuição dos Resíduos com Curva Normal ajustada<br><sup>{modelo} | Assimetria de {round(residuos.skew(), 2)}</sup>",
            text=f"{titulo}<br><sup>{subtitulo}</sup>",
            font=dict(family="system-ui", size=26)
        ),
        xaxis=dict(
            title     = "Resíduos",
            titlefont = dict(size=20),  # Tamanho da fonte do título do eixo X
            tickfont  = dict(size=16),  # Tamanho da fonte dos ticks do eixo X
        ),
        yaxis=dict(
            title     = "Contagem",  # Eixo y1 (à esquerda)
            titlefont = dict(size=20),  # Tamanho da fonte do título do eixo Y1
            tickfont  = dict(size=16),  # Tamanho da fonte dos ticks do eixo Y1
            showgrid  = False,
        ),
        yaxis2=dict(
            title      = "Densidade",  # Eixo y2 (à direita)
            titlefont  = dict(size=20),  # Tamanho da fonte do título do eixo Y2
            tickfont   = dict(size=16),  # Tamanho da fonte dos ticks do eixo Y2
            overlaying = 'y',  # Sobrepõe ao y1
            side       = 'right',
        ),
        showlegend=False
    )

    fig = go.Figure(data=[hist_data, normal_curve], layout=layout)

    fig.show()
# ============================================================================================ #
def box_plot_residuos(residuos, cor_box_linhas, cor_outliers, modelo):
    box_plot = go.Box(
        y         = residuos,
        fillcolor = cor_box_linhas,
        marker    = dict(
            color=cor_outliers,  # Cor dos marcadores (outliers)
            # size=8    # Tamanho dos marcadores
        ),
        line=dict(
            color=cor_box_linhas,  # Cor das linhas da caixa, whiskers, etc.
            # width=2        # Espessura das linhas
        ),
        name      = 'Resíduos',
        boxpoints = 'outliers'  # Mostrar apenas os outliers
    )

    layout = go.Layout(
        width  = 550,
        height = 1150,
        title  = dict(
            text=f'Box Plot dos Resíduos<br><sup>{modelo}</sup>',
            font=dict(family="system-ui", size=26)
        ),
        xaxis  = dict(
            title='',
            titlefont=dict(size=20),
            tickfont=dict(size=20),
        ),
        yaxis  = dict(
            title='Valor do Resíduo',
            titlefont=dict(size=20),
            tickfont=dict(size=16),
            showgrid=False,
        ),
        showlegend = False
    )

    fig = go.Figure(data=[box_plot], layout=layout)
    fig.show()
# ============================================================================================ #
# def plot_cv(
#         fh           : int,
#         df_merged    : pd.DataFrame,
#         df_resultado : pd.DataFrame,
#         regressor    : str,
#         data_inicio  : str,
#         n_decimais   : int,
#         titulo_plot  : str,
#         pasta_dstn   : str,
#         salvar       : bool = False,
# ) -> None:

#     fig = go.Figure()

#     fig = make_subplots(
#         rows=2,
#         cols=1,
#         vertical_spacing=0.2,
#         specs=[
#             [{"type": "scatter"}],
#             [{"type": "table"}]
#         ],
#     )

#     fig.add_trace(
#         go.Scatter(
#             x=df_merged[df_merged['ds'] >= data_inicio]["ds"],
#             y=df_merged[df_merged['ds'] >= data_inicio]["y"],
#             mode="lines+markers",
#             name="Observado",
#             line=dict(color="#000000", width=2),
#         ),
#         row=1,
#         col=1,
#     )

#     fig.add_trace(
#         go.Scatter(
#             x=df_merged[df_merged['ds'] >= data_inicio]["ds"],
#             y=df_merged[df_merged['ds'] >= data_inicio][regressor],
#             mode="lines+markers",
#             name="Previsão",
#             line=dict(color="red"),
#         ),
#         row=1,
#         col=1,
#     )

#     fig.add_trace(
#         go.Table(
#             header=dict(
#                 values=[
#                     "MAPE",
#                     "RMSE",
#                     "PBIAS (%)",
#                     "DRV",
#                 ],
#                 font=dict(size=18),
#                 align="center"
#             ),
#             cells=dict(
#                 values=[
#                     round(mape(df_resultado.y, df_resultado[regressor]), n_decimais),
#                     round(rmse(df_resultado.y, df_resultado[regressor]), n_decimais),
#                     round(percentage_bias(df_resultado.y, df_resultado[regressor]), n_decimais),
#                     round(deviation_runoff_volume(df_resultado.y, df_resultado[regressor]), n_decimais),
#                 ],
#                 font=dict(size=18),
#                 height=30,
#                 align="left",
#             ),
#         ),
#         row=2,
#         col=1,
#     )

#     fig.update_yaxes(
#         zerolinecolor="black",
#         mirror=True,
#         ticks="outside",
#         showline=True,
#         linecolor="black",
#         title=dict(text="Vazão (m³/s)", font=dict(family="system-ui", size=20))
#     )

#     fig.update_xaxes(
#         mirror=True,
#         ticks="outside",
#         showline=True,
#         linecolor="black",
#         title=dict(text="Período", font=dict(family="system-ui", size=20))
#     )

#     fig.update_layout(
#         width=1150,
#         height=700,
#         hovermode="x unified",
#         plot_bgcolor="#c8d4e3",
#         xaxis=dict(tickfont=dict(family="system-ui", size=16)),
#         yaxis=dict(tickfont=dict(family="system-ui", size=16)),
#         title=dict(text=titulo_plot, font=dict(family="system-ui", size=26))
#     )

#     if salvar:
#         if not os.path.exists(pasta_dstn):
#             os.makedirs(pasta_dstn)

#         now = datetime.now()
#         fig.write_image(pasta_dstn + f"/cv_{regressor}_fh{fh}_{now.strftime('%Y-%m-%d_%H-%M-%S')}.png")
#     else:
#         fig.show()
# ============================================================================================ #
# def cria_dataframe_futuro(
#     df_futr: pd.DataFrame,
#     df_train: pd.DataFrame,
#     df_test: pd.DataFrame,
#     tp_valor: str,
#     n_lags: int,
#     date_features: list,
#     cols: list,
# ) -> pd.DataFrame:
    
#     """
#         tp_valor == "ultimo": # Usa o último valor conhecido
#         tp_valor == "media":  # Usa o valor médio de cada coluna vazão
#         tp_valor == "ml":     # Usa um modelo XGBoost para gerar previsão futuras das vazões auxiliares
#     """

#     if tp_valor == "ultimo":  # Usa o último valor conhecido
#         for c in cols:
#             df_futr[c] = df_train[c].iat[-1]

#     elif tp_valor == "media":  # Usa o valor médio de cada coluna vazão
#         for c in cols:
#             df_futr[c] = df_train[c].mean()

#     elif tp_valor == "ml":
#         from xgboost import XGBRegressor

#         for c in cols:
#             fcst = mlf.MLForecast(
#                 models=XGBRegressor(seed=SEED),
#                 freq="D",
#                 lags=[i + 1 for i in range(n_lags)],
#                 date_features=date_features,
#             )

#             df_temp = df_train[["ds", "unique_id", c]]

#             fcst.fit(
#                 df_temp,
#                 id_col="unique_id",
#                 time_col="ds",
#                 target_col=c,
#                 static_features=[],
#             )

#             df_preds = fcst.predict(h=len(df_futr)).reset_index()  # macetasso pra não dar erro de index
#             df_futr[c] = df_preds["XGBRegressor"]

#     else:
#         raise Exception("Opção inválida! (ultimo | media | ml)")

#     df_futr = pd.merge(
#         left=df_futr,
#         right=df_test.drop(columns=cols + ["y"]),
#         on=["ds", "unique_id"],
#         how="left",
#     )

#     return df_futr
# ============================================================================================ #
# def distribuicao_dados(
#     df_original: pd.DataFrame,
#     df_1: pd.DataFrame,
#     nome_1: str,
#     df_2: pd.DataFrame,
#     nome_2: str,
#     salvar: bool = False,
# ) -> None:

#     # cols = np.asarray(df_original.drop(columns=["ds", "unique_id"]).columns)
#     cols = df_original.drop(columns=["ds"]).columns.to_list()

#     for c in cols:
#         fig = go.Figure()

#         fig.add_trace(
#             go.Box(
#                 y=df_original[c].values,
#                 name="original",
#                 marker_color="darkblue",
#                 jitter=0.5,
#                 pointpos=-2,
#                 boxpoints="all",
#                 boxmean="sd",
#             )
#         )

#         fig.add_trace(
#             go.Box(
#                 y=df_1[c].values,
#                 name=nome_1,
#                 marker_color="coral",
#                 jitter=0.5,
#                 pointpos=-2,
#                 boxpoints="all",
#                 boxmean="sd",
#             )
#         )

#         fig.add_trace(
#             go.Box(
#                 y=df_2[c].values,
#                 name=nome_2,
#                 marker_color="olive",
#                 jitter=0.5,
#                 pointpos=-2,
#                 boxpoints="all",
#                 boxmean="sd",
#             )
#         )

#         fig.update_xaxes(
#             mirror=True,
#             ticks="outside",
#             showline=True,
#             linecolor="black",
#         )

#         fig.update_yaxes(
#             zerolinecolor="black",
#             mirror=True,
#             ticks="outside",
#             showline=True,
#             linecolor="black",
#         )

#         fig.update_layout(
#             width=1500,
#             height=1000,
#             plot_bgcolor="#c8d4e3",
#             title=dict(
#                 text="Distribuição {c}".format(c=c),
#                 font=dict(family="system-ui", size=24),
#             ),
#         )

#         if salvar:
#             fig.write_image(
#                 pasta_resultados+"aed/distribuicao_dados_{}.png".format(c)
#             )
#         else:
#             fig.show()
# ============================================================================================ #
# def estacionariedade(
#     df : pd.DataFrame,
#     sp : int
# ) -> None:

#     # Avaliar a estacionariedade de cada uma das séries e a sazonalidade (se houver)
#     # Existindo sazonalidade, qual a lag (ou quais lags) se encaixam nesta sazonalidade
#     # cols = df.drop(columns=["ds", "unique_id"]).columns.to_list()
#     cols = df.drop(columns=["ds"]).columns.to_list()
#     for c in cols:
#         ts = df[c]
#         sty_est = StationarityADF()
#         sty_est.fit(ts)
#         print(c, sty_est.get_fitted_params()["stationary"])

#         # Este teste de sazonalidade deve ser aplicado a séries estacionárias.
#         # Se precisar tornar uma série em estacionária, tem de aplicar diferenciação antes.
#         if sty_est.get_fitted_params()["stationary"]:
#             sp_est = SeasonalityACF( # Minha intenção é ter certeza de que existe sazonalidade anual (365 dias)
#                 candidate_sp=sp,
#                 nlags=len(df[c])
#             )
#             sp_est.fit(ts)
#             sp_est.get_fitted_params()
#             print(c, sp_est.get_fitted_params()["sp_significant"])

# Carregando e imputando dados

In [4]:
df = pd.read_excel(
    io          = "./arquivos_finais/baixo_rio_grande_final.xlsx",
    sheet_name  = 0,
    index_col   = 0,
    header      = 0,
    parse_dates = ["Data"],
)

In [6]:
df

,t_cv_62020080,t_vz_62020080,t_cv_61998080
Data,,,
2014-12-30,0,NaN,NaN
2014-12-31,0,NaN,NaN
2015-01-01,0,NaN,NaN
2015-01-02,0,NaN,NaN
2015-01-03,0,NaN,NaN
...,...,...,...
2023-12-27,0,3252.283333,0.0
2023-12-28,0,3698.487500,0.0
2023-12-29,0,3620.600000,0.0


In [7]:
coluna_alvo = "t_vz_62020080"

In [8]:
# Deixando o DataFrame no padrão que a lib MLForecast obriga
df = df.reset_index()
df = df.rename(columns={
    "Data"       : "ds",
    coluna_alvo : "y"
    })

df

,ds,t_cv_62020080,y,t_cv_61998080
0,2014-12-30,0,NaN,NaN
1,2014-12-31,0,NaN,NaN
2,2015-01-01,0,NaN,NaN
3,2015-01-02,0,NaN,NaN
4,2015-01-03,0,NaN,NaN
...,...,...,...,...
3284,2023-12-27,0,3252.283333,0.0
3285,2023-12-28,0,3698.487500,0.0
3286,2023-12-29,0,3620.600000,0.0
3287,2023-12-30,0,3672.120833,0.0


In [9]:
# Só reordenando a posição das colunas pra ficar mais fácil de ler
df = df[[
    "ds",
    "t_cv_62020080",
    "t_cv_61998080",
    "y",
]]

df

,ds,t_cv_62020080,t_cv_61998080,y
0,2014-12-30,0,NaN,NaN
1,2014-12-31,0,NaN,NaN
2,2015-01-01,0,NaN,NaN
3,2015-01-02,0,NaN,NaN
4,2015-01-03,0,NaN,NaN
...,...,...,...,...
3284,2023-12-27,0,0.0,3252.283333
3285,2023-12-28,0,0.0,3698.487500
3286,2023-12-29,0,0.0,3620.600000
3287,2023-12-30,0,0.0,3672.120833


In [10]:
plot_serie_temporal(
    dataset=df,
    coluna="t_cv_61998080",
    tp_coluna="chuva",
    plot_title="Série temporal incompleta da estação {}".format("t_cv_61998080"),
    line_color="darkgreen",
    short_name="t_cv_61998080",
    pasta_resultados=pasta_resultados+"aed/",
    salvar=SALVAR_PLOTS
)

In [11]:
# Verificando dados faltantes

print("Quantidade de dados faltantes:\n{}".format(df[df.select_dtypes(include=['float64', 'int64']).columns].isna().sum()))
print("---")
print("Percentual de dados faltantes:\n{}".format((100 * df[df.select_dtypes(include=['float64', 'int64']).columns].isna().sum()) / len(df)))

Quantidade de dados faltantes:
t_cv_62020080       0
t_cv_61998080     169
y                2099
dtype: int64
---
Percentual de dados faltantes:
t_cv_62020080     0.00000
t_cv_61998080     5.13834
y                63.81879
dtype: float64


In [12]:
# Preenchendo com dados médios dos últimos 3 anos
num_cols = df.select_dtypes(include=['float64', 'int64']).columns
df_sazonal = fill_with_previous_years_average(df, num_cols)
df_sazonal

,ds,t_cv_62020080,t_cv_61998080,y
0,2014-12-30,0,NaN,NaN
1,2014-12-31,0,NaN,NaN
2,2015-01-01,0,NaN,NaN
3,2015-01-02,0,NaN,NaN
4,2015-01-03,0,NaN,NaN
...,...,...,...,...
3284,2023-12-27,0,0.0,3252.283333
3285,2023-12-28,0,0.0,3698.487500
3286,2023-12-29,0,0.0,3620.600000
3287,2023-12-30,0,0.0,3672.120833


In [13]:
# Verificando dados faltantes

print("Quantidade de dados faltantes:\n{}".format(
    df_sazonal[df_sazonal.select_dtypes(include=['float64', 'int64']).columns].isna().sum()
))
print("---")
print("Percentual de dados faltantes:\n{}".format(
    (100 * df_sazonal[df_sazonal.select_dtypes(include=['float64', 'int64']).columns].isna().sum()) / len(df_sazonal)
))

Quantidade de dados faltantes:
t_cv_62020080       0
t_cv_61998080     169
y                2066
dtype: int64
---
Percentual de dados faltantes:
t_cv_62020080     0.000000
t_cv_61998080     5.138340
y                62.815445
dtype: float64


## Preenchendo com o KNNImputer

O que resta de dado faltante preenche com o KNNImputer

In [14]:
# Ainda precisa preencher dados faltates. Vou aplicar o KNNImputer pra finalizar isso.

imputer = KNNImputer(
    n_neighbors=7,
    weights="distance" # vizinhos mais próximos têm mais influência
)

df_knn = pd.DataFrame(
    data=imputer.fit_transform(df_sazonal.drop(columns=["ds"])),
    columns=df_sazonal.drop(columns=["ds"]).columns,
)

df_knn = pd.DataFrame(
    data=df_knn,
    columns=df_sazonal.drop(columns=["ds"]).columns
)

df_knn = pd.concat(
    [df_sazonal[["ds"]], df_knn],
    axis=1
)

df_knn

,ds,t_cv_62020080,t_cv_61998080,y
0,2014-12-30,0.0,5.085714,3811.803571
1,2014-12-31,0.0,5.085714,3811.803571
2,2015-01-01,0.0,5.085714,3811.803571
3,2015-01-02,0.0,5.085714,3811.803571
4,2015-01-03,0.0,5.085714,3811.803571
...,...,...,...,...
3284,2023-12-27,0.0,0.000000,3252.283333
3285,2023-12-28,0.0,0.000000,3698.487500
3286,2023-12-29,0.0,0.000000,3620.600000
3287,2023-12-30,0.0,0.000000,3672.120833


In [15]:
# Quantos zeros existem nas colunas
target = ['y']

chuvas = [
    "t_cv_62020080",
    "t_cv_61998080",
]

print("Target")
print(target[0], (df_knn[target[0]] == 0).sum())

print("Chuvas")
for c in chuvas:
    print(c, (df_knn[c] == 0).sum())

Target
y 0
Chuvas
t_cv_62020080 3289
t_cv_61998080 2303


In [16]:
# A coluna "t_cv_62020080" está completamente zerada. Vou remover.
df_knn = df_knn.drop(columns=["t_cv_62020080"]).copy()

In [17]:
chuvas = [
    "t_cv_61998080",
]

# Análise Exploratória dos Dados

In [18]:
# Adicionando atributos categóricos de Data

atributos_categoricos = ['estacao', 'month', 'dayofyear', 'week', 'quarter']
df_knn = gerar_atributos_data(
    df        = df_knn,
    atributos = atributos_categoricos,
    col_data  = "ds"
)

df_knn

,ds,t_cv_61998080,y,estacao,month,dayofyear,week,quarter
0,2014-12-30,5.085714,3811.803571,1,12,364,1,4
1,2014-12-31,5.085714,3811.803571,1,12,365,1,4
2,2015-01-01,5.085714,3811.803571,1,1,1,1,1
3,2015-01-02,5.085714,3811.803571,1,1,2,1,1
4,2015-01-03,5.085714,3811.803571,1,1,3,1,1
...,...,...,...,...,...,...,...,...
3284,2023-12-27,0.000000,3252.283333,1,12,361,52,4
3285,2023-12-28,0.000000,3698.487500,1,12,362,52,4
3286,2023-12-29,0.000000,3620.600000,1,12,363,52,4
3287,2023-12-30,0.000000,3672.120833,1,12,364,52,4


In [19]:
df_bar = df_knn.rename(columns={target[0] : coluna_alvo})

fig = px.histogram(
    data_frame = df_bar[chuvas],
    marginal   = "box",
    barmode    = 'overlay',
    opacity    = 0.75,
    width      = 1150,
    height     = 550,
    color_discrete_sequence = px.colors.qualitative.T10
)

fig.update_layout(
    xaxis            = dict(tickfont=dict(family="system-ui", size=16)),
    xaxis_title_text = "valor",
    xaxis_title_font = dict(family="system-ui", size=20),
    yaxis            = dict(tickfont=dict(family="system-ui", size=16)),
    yaxis_title_text = "contagem",
    yaxis_title_font = dict(family="system-ui", size=20),
)

fig.show()

In [20]:
fig = px.histogram(
    data_frame = np.log1p(df_bar[chuvas]),
    marginal   = "box",
    barmode    = 'overlay',
    opacity    = 0.75,
    width      = 1150,
    height     = 550,
    color_discrete_sequence = px.colors.qualitative.T10
)

fig.update_layout(
    xaxis            = dict(tickfont=dict(family="system-ui", size=16)),
    xaxis_title_text = "valor",
    xaxis_title_font = dict(family="system-ui", size=20),
    yaxis            = dict(tickfont=dict(family="system-ui", size=16)),
    yaxis_title_text = "contagem",
    yaxis_title_font = dict(family="system-ui", size=20),
)

fig.show()

In [21]:
media_res = np.mean(df_bar[coluna_alvo].values)
std_res   = np.std(df_bar[coluna_alvo].values)

hist_data = go.Histogram(
    x=df_bar[coluna_alvo],
    nbinsx=50,
    # histnorm ='probability density',  # Normaliza o histograma para densidade de probabilidade
    marker=dict(color='blue'),
    yaxis='y1',
    opacity=0.75,
    name='Histograma'
)

xmin, xmax = df_bar[coluna_alvo].min(), df_bar[coluna_alvo].max()
x = np.linspace(xmin, xmax, 100)
y = norm.pdf(x, media_res, std_res)

normal_curve = go.Scatter(
    x=x,
    y=y,
    mode='lines',
    line=dict(color='darkorange', width=2),
    yaxis='y2',
    name='Curva Normal'
)

layout = go.Layout(
    width=1150,
    height=550,
    title=dict(
        text=f"Variável {coluna_alvo}<br><sup>Histograma e Curva Normal ajustada - série original | Assimetria de {round(df_bar[coluna_alvo].skew(), 2)}</sup>",
        font=dict(family="system-ui", size=26)
    ),
    xaxis=dict(
        title     = "Valor",
        titlefont = dict(size=20),  # Tamanho da fonte do título do eixo X
        tickfont  = dict(size=16),  # Tamanho da fonte dos ticks do eixo X
    ),
    yaxis=dict(
        title     = "Contagem",  # Eixo y1 (à esquerda)
        titlefont = dict(size=20),  # Tamanho da fonte do título do eixo Y1
        tickfont  = dict(size=16),  # Tamanho da fonte dos ticks do eixo Y1
        showgrid  = False,
    ),
    yaxis2=dict(
        title      = "Densidade",  # Eixo y2 (à direita)
        titlefont  = dict(size=20),  # Tamanho da fonte do título do eixo Y2
        tickfont   = dict(size=16),  # Tamanho da fonte dos ticks do eixo Y2
        overlaying = 'y',  # Sobrepõe ao y1
        side       = 'right',
    ),
    showlegend=False
)

fig = go.Figure(data=[hist_data, normal_curve], layout=layout)

fig.show()

In [22]:
log_data = pd.DataFrame(np.log1p(df_bar[coluna_alvo]))

media_res = np.mean(log_data[coluna_alvo].values)
std_res   = np.std(log_data[coluna_alvo].values)

hist_data = go.Histogram(
    x=log_data[coluna_alvo],
    nbinsx=50,
    # histnorm ='probability density',  # Normaliza o histograma para densidade de probabilidade
    marker=dict(color='blue'),
    yaxis='y1',
    opacity=0.75,
    name='Histograma'
)

xmin, xmax = log_data[coluna_alvo].min(), log_data[coluna_alvo].max()
x = np.linspace(xmin, xmax, 100)
y = norm.pdf(x, media_res, std_res)

normal_curve = go.Scatter(
    x=x,
    y=y,
    mode='lines',
    line=dict(color='darkorange', width=2),
    yaxis='y2',
    name='Curva Normal'
)

layout = go.Layout(
    width=1150,
    height=550,
    title=dict(
        text=f"Variável {coluna_alvo}<br><sup>Histograma e Curva Normal ajustada - série log-transformada | Assimetria de {round(log_data[coluna_alvo].skew(), 2)}</sup>",
        font=dict(family="system-ui", size=26)
    ),
    xaxis=dict(
        title     = "Valor",
        titlefont = dict(size=20),  # Tamanho da fonte do título do eixo X
        tickfont  = dict(size=16),  # Tamanho da fonte dos ticks do eixo X
    ),
    yaxis=dict(
        title     = "Contagem",  # Eixo y1 (à esquerda)
        titlefont = dict(size=20),  # Tamanho da fonte do título do eixo Y1
        tickfont  = dict(size=16),  # Tamanho da fonte dos ticks do eixo Y1
        showgrid  = False,
    ),
    yaxis2=dict(
        title      = "Densidade",  # Eixo y2 (à direita)
        titlefont  = dict(size=20),  # Tamanho da fonte do título do eixo Y2
        tickfont   = dict(size=16),  # Tamanho da fonte dos ticks do eixo Y2
        overlaying = 'y',  # Sobrepõe ao y1
        side       = 'right',
    ),
    showlegend=False
)

fig = go.Figure(data=[hist_data, normal_curve], layout=layout)

fig.show()

## Séries Temporais

In [22]:
plot_serie_temporal(
    dataset=df,
    coluna="y",
    tp_coluna="vazao",
    plot_title="Série temporal incompleta da estação {}".format("t_vz_62020080"),
    line_color="darkred",
    short_name="62020080",
    pasta_resultados=pasta_resultados+"aed/",
    salvar=SALVAR_PLOTS
)

In [23]:
for c in chuvas:
    plot_serie_temporal(
        dataset=df_knn,
        coluna=c,
        tp_coluna="chuva",
        plot_title="Série temporal completa da estação {}".format(c),
        line_color="darkgreen",
        short_name=c,
        pasta_resultados=pasta_resultados+"aed/",
        salvar=SALVAR_PLOTS
    )

In [24]:
for t in target:
    plot_serie_temporal(
        dataset=df_knn,
        coluna=t,
        tp_coluna="vazao",
        plot_title="Série temporal completa da estação {}".format("t_vz_62020080"),
        line_color="darkred",
        short_name='t_vz_62020080',
        pasta_resultados=pasta_resultados+"aed/",
        salvar=SALVAR_PLOTS
    )

## Decomposição das Séries Temporais

A decomposição das séries temporais ajuda a detectar padrões (tendência, sazonalidade) e identificar outras informações que podem ajudar na interpretação do que está acontecendo.

In [25]:
df_decomp = df_knn.rename(columns={'y': coluna_alvo})

decomp_series(
    df           = df_decomp.drop(columns=atributos_categoricos),
    observado    = False,
    tendencia    = False,
    sazonalidade = True,
    residuo      = False,
    cor_linha    = "darkorange",
    salvar       = SALVAR_PLOTS,
)

## Estacionariedade

In [26]:
# estacionariedade(
#     df=df_knn.drop(columns=atributos_categoricos),
#     sp=range(1, 367)
# )

t_cv_61998080 True
t_cv_61998080 [  6   1   5   7  11  13  14  12  24   8  18   2   9  15  20  17  26  10
  25  22  16  32  19  33  23  21   4  31   3  30  34  27  38  48  35  39
  29  43  47  37  36 320  46]
y True
y [ 1  2  3  4  5  6  7  8 28  9 29 21 10 27 14 30 22 26 25 11 13 31 20 12
 23 15 24 35 32 34 33 16 19 17 18 36 37 56 38 55 57 42 49 39 41 54 40 53
 58 63 52 50 48 59 51 43 62 60 61 47 46 45 44 64 70 84 65 77 66 83 69 71
 78 76 85 67 68 91]


## Correlação entre as séries

In [36]:
mapa_correlacao(
    df     = df_knn.drop(columns=atributos_categoricos),
    medida = "pearson",
    salvar = SALVAR_PLOTS
)

In [28]:
# Preferi jogar os dados alterados para um novo DataFrame porque se precisar voltar no DataFrame inicial, não precisará regarregar o arquivo
df_aux = df_knn.copy()

df_aux = df_aux.set_index(pd.to_datetime(df_aux['ds'])).drop(columns=['ds']).copy()
df_aux = df_aux.asfreq("D")
df_aux

,t_cv_61998080,y,estacao,month,dayofyear,week,quarter
ds,,,,,,,
2014-12-30,5.085714,3811.803571,1,12,364,1,4
2014-12-31,5.085714,3811.803571,1,12,365,1,4
2015-01-01,5.085714,3811.803571,1,1,1,1,1
2015-01-02,5.085714,3811.803571,1,1,2,1,1
2015-01-03,5.085714,3811.803571,1,1,3,1,1
...,...,...,...,...,...,...,...
2023-12-27,0.000000,3252.283333,1,12,361,52,4
2023-12-28,0.000000,3698.487500,1,12,362,52,4
2023-12-29,0.000000,3620.600000,1,12,363,52,4


## Análise de Autocorrelação

In [29]:
# Me interessa saber a sazonalidade da variável-alvo, a vazão
cria_plot_correlacao(
    serie     = df_aux[target],
    n_lags    = 1000,
    coluna    = coluna_alvo,
    plot_pacf = False,
    salvar    = SALVAR_PLOTS
)

In [30]:
cria_plot_correlacao(
    serie     = df_aux[target],
    n_lags    = 15,
    plot_pacf = True,
    coluna    = coluna_alvo,
    salvar    = SALVAR_PLOTS
)

## Relação entre as variáveis

In [31]:
# for c in chuvas:
#     fig = go.Figure()

#     fig.add_trace(
#         go.Scatter(
#             x=df_aux[c],
#             y=df_aux[target[0]],
#             mode="markers",
#             line=dict(color="green"),
#             hovertemplate="eixo_x: %{x}<br>eixo_y: %{y}</br><extra></extra>",
#             showlegend=False,
#         )
#     )

#     fig.update_yaxes(
#         title=dict(
#             text=df_aux[target[0]].name,
#             font=dict(family="system-ui", size=18)
#         ),
#         zerolinecolor="black",
#         showspikes=True,
#         mirror=True,
#         ticks="outside",
#         showline=True,
#         linecolor="black",
#     )

#     fig.update_xaxes(
#         title=dict(
#             text=df_aux[c].name,
#             font=dict(family="system-ui", size=18)
#         ),
#         zerolinecolor="black",
#         showspikes=True,
#         mirror=True,
#         ticks="outside",
#         showline=True,
#         linecolor="black",
#     )

#     fig.update_layout(
#         width=1500,
#         height=700,
#         hovermode="closest",
#         plot_bgcolor="#c8d4e3",
#         title=dict(
#             text="Relação entre as variáveis {t} e '{c}'".format(t=target[0], c=c),
#             font=dict(family="system-ui", size=24),
#         ),
#     )

#     if SALVAR_PLOTS:
#         fig.write_image(
#             pasta_resultados+"aed/relacao_{t}_{c}.png".format(t=target[0], c=c)
#         )
#     else:
#         fig.show()

## Granger-causality

In [32]:
# from statsmodels.tsa.stattools import grangercausalitytests

# # vazoes = ["t_vz_56990850", "t_vz_56990005", "c_vz_56989400", "c_vz_56989900", "c_vz_56990000"]
# # chuvas = ["c_cv_01941010", "c_cv_01941004", "c_cv_01941006", "t_cv_56990005"]

# df_granger = pd.DataFrame()
# df_granger = df_aux.drop(columns=["ds", "unique_id"]).diff(1)  # aplica essa diferenciação pra remover qq efeito de tendência
# df_granger = df_granger.dropna()

# grangercausalitytests(
#     x=df_granger[["y", "t_vz_56990850"]].tail(30),
#     maxlag=7,
#     verbose=True
# )

# Variáveis globais

In [33]:
df_aux.describe()

,t_cv_61998080,y,estacao,month,dayofyear,week,quarter
count,3289.000000,3289.000000,3289.000000,3289.000000,3289.000000,3289.0,3289.000000
mean,3.131868,3405.266737,2.496808,6.526604,183.221648,26.620553,2.509577
std,13.922053,873.879347,1.110147,3.450727,105.509382,15.073345,1.117483
min,0.000000,1603.583333,1.000000,1.000000,1.000000,1.0,1.000000
25%,0.000000,3170.078571,2.000000,4.000000,92.000000,14.0,2.000000
50%,0.000000,3170.078571,3.000000,7.000000,183.000000,27.0,3.000000
75%,0.600000,3575.173913,3.000000,10.000000,275.000000,40.0,4.000000
max,330.400000,11939.487500,4.000000,12.000000,366.000000,53.0,4.000000


In [34]:
look_back = 7 # Lags a serem utilizadas. Uma semana passada.

fh_v = [1, 3, 7, 15]  # Horizonte de Previsão (como a frequência dos dados é diária, isso significa "fh" dias)

fh_artigo = [1, 3, 7]  # Horizonte de Previsão inspirado no artigo da Alemanha

intervalos_previsao = [90]
# No gráfico será mostrado apenas os níveis que estiverem aqui.
# Deve ser posto na ordem inversa, ou seja, do maior pro menor nível.
# intervalos_previsao_plotar = ["95", "80"]
intervalos_previsao_plotar = ["90"]

colunas_chuva = chuvas
# colunas_vazao = vazoes

# Cenário de experimentação
cenario1 = "sem_chuva"
cenario2 = "com_chuva"

In [35]:
# Versão com transformação logarítmica dos dados

df_aux_log = df_aux.copy()
for c in chuvas:
    df_aux_log[c] = np.log1p(df_aux_log[c])

df_aux_log[target] = np.log1p(df_aux_log[target])

df_aux_log

,t_cv_61998080,y,estacao,month,dayofyear,week,quarter
ds,,,,,,,
2014-12-30,1.805944,8.246120,1,12,364,1,4
2014-12-31,1.805944,8.246120,1,12,365,1,4
2015-01-01,1.805944,8.246120,1,1,1,1,1
2015-01-02,1.805944,8.246120,1,1,2,1,1
2015-01-03,1.805944,8.246120,1,1,3,1,1
...,...,...,...,...,...,...,...
2023-12-27,0.000000,8.087420,1,12,361,52,4
2023-12-28,0.000000,8.215950,1,12,362,52,4
2023-12-29,0.000000,8.194671,1,12,363,52,4


# Separação dos dados

In [69]:
# Os dados do ano de 2023 serão um DataFrame especial

df_2023 = df_aux[df_aux.index.year == 2023].copy()
df_2023

,t_cv_61998080,y,estacao,month,dayofyear,week,quarter
ds,,,,,,,
2023-01-01,0.0,2710.783333,1,1,1,52,1
2023-01-02,26.8,3686.033333,1,1,2,1,1
2023-01-03,9.4,3349.416667,1,1,3,1,1
2023-01-04,116.2,3760.529167,1,1,4,1,1
2023-01-05,40.6,3275.383333,1,1,5,1,1
...,...,...,...,...,...,...,...
2023-12-27,0.0,3252.283333,1,12,361,52,4
2023-12-28,0.0,3698.487500,1,12,362,52,4
2023-12-29,0.0,3620.600000,1,12,363,52,4


In [70]:
# Vou deixar um DataFrame já com a Transformação Logarítmica

df_2023_log = df_2023.copy()
for c in chuvas:
    df_2023_log[c] = np.log1p(df_2023_log[c])

df_2023_log[target] = np.log1p(df_2023_log[target])

df_2023_log

,t_cv_61998080,y,estacao,month,dayofyear,week,quarter
ds,,,,,,,
2023-01-01,0.000000,7.905362,1,1,1,52,1
2023-01-02,3.325036,8.212577,1,1,2,1,1
2023-01-03,2.341806,8.116840,1,1,3,1,1
2023-01-04,4.763882,8.232581,1,1,4,1,1
2023-01-05,3.728100,8.094495,1,1,5,1,1
...,...,...,...,...,...,...,...
2023-12-27,0.000000,8.087420,1,12,361,52,4
2023-12-28,0.000000,8.215950,1,12,362,52,4
2023-12-29,0.000000,8.194671,1,12,363,52,4


In [71]:
df_dados = df_aux.drop(index=df_2023.index).copy()
df_dados

,t_cv_61998080,y,estacao,month,dayofyear,week,quarter
ds,,,,,,,
2014-12-30,5.085714,3811.803571,1,12,364,1,4
2014-12-31,5.085714,3811.803571,1,12,365,1,4
2015-01-01,5.085714,3811.803571,1,1,1,1,1
2015-01-02,5.085714,3811.803571,1,1,2,1,1
2015-01-03,5.085714,3811.803571,1,1,3,1,1
...,...,...,...,...,...,...,...
2022-12-27,41.000000,3618.329167,1,12,361,52,4
2022-12-28,0.200000,3600.975000,1,12,362,52,4
2022-12-29,97.800000,3739.187500,1,12,363,52,4


In [72]:
# Vou deixar um DataFrame já com a Transformação Logarítmica

df_dados_log = df_dados.copy()
for c in chuvas:
    df_dados_log[c] = np.log1p(df_dados_log[c])

df_dados_log[target] = np.log1p(df_dados_log[target])

df_dados_log

,t_cv_61998080,y,estacao,month,dayofyear,week,quarter
ds,,,,,,,
2014-12-30,1.805944,8.246120,1,12,364,1,4
2014-12-31,1.805944,8.246120,1,12,365,1,4
2015-01-01,1.805944,8.246120,1,1,1,1,1
2015-01-02,1.805944,8.246120,1,1,2,1,1
2015-01-03,1.805944,8.246120,1,1,3,1,1
...,...,...,...,...,...,...,...
2022-12-27,3.737670,8.194044,1,12,361,52,4
2022-12-28,0.182322,8.189238,1,12,362,52,4
2022-12-29,4.593098,8.226891,1,12,363,52,4


In [73]:
plot_divisao_treino_teste(
    df_treino = df_dados,
    df_teste  = df_2023,
    col_data  = None,
    col_plot  = target[0],
    index_ds  = True,
    salvar    = SALVAR_PLOTS
)

# sem chuva, com dados categóricos

## StatsForecast

### Baseline - SeasonalNaive

In [74]:
for f in fh_v:
    # Realiza a separação dos dados neste ponto para cada horizonte de previsão desejado
    treino = df_dados.iloc[:-f, :].copy()
    teste = df_dados.iloc[-f:, :].copy()

    # Exigência da lib MLForecast
    treino = treino.reset_index()
    teste = teste.reset_index()
    treino["unique_id"] = 1
    teste["unique_id"] = 1

    modelo = SeasonalNaive(season_length=365) # Sazonalidade de 365 dias

    stfc = StatsForecast(
        df     = treino.drop(columns=colunas_chuva),
        models = [modelo],
        freq   = "D",
        n_jobs = NJOBS
    )

    df_futr = teste.drop(columns=target+colunas_chuva).copy()
    y_pred = stfc.forecast(
        h     = f,
        X_df  = df_futr, # passa apenas os atributos categóricos
        level = intervalos_previsao
    )

    df_resultado = pd.merge(
        left  = y_pred,
        right = teste[['ds', 'unique_id', 'y']],
        how   = "left",
        on    = ['ds', 'unique_id']
    )

    plot_resultados(
        df_merged  = df_resultado,
        modelo     = modelo.alias,
        nome_curto = "SN",
        fh         = f,
        titulo     = "{md} (fh={fh}) ({c})".format(md=modelo.alias, fh=f, c=cenario1),
        niveis     = intervalos_previsao_plotar,
        cores      = ["green"],
        pasta_dstn = pasta_resultados+cenario1,
        salvar     = SALVAR_PLOTS
    )

## SKForecast

### LinearRegression, LightGBM e CatBoost

A partir da baseline estipulada pelo SeasonalNaive, estes modelos são testados para ver se melhoram os resultados

In [75]:
for f in fh_v:
    treino = df_dados_log.iloc[:-f, :].copy() 
    teste = df_dados_log.iloc[-f:, :].copy()

    modelos = {
        "LinearRegression"  : [LinearRegression(), "LR"],
        "CatBoostRegressor" : [CatBoostRegressor(random_seed=SEED, cat_features=atributos_categoricos, verbose=False, allow_writing_files=False), "CB"],
        "LGBMRegressor"     : [LGBMRegressor(random_state=SEED, verbose=-1), "LGBM"]
    }

    # Isso é exclusivo para o LightGBM, passado no momento do 'fit'.
    lgbm_fit_arg = {"categorical_feature": atributos_categoricos}

    for chave, item in modelos.items():
        forecaster = ForecasterAutoreg(
            regressor  = item[0],
            lags       = look_back,
            fit_kwargs = lgbm_fit_arg if chave == "LGBMRegressor" else None
        )

        # Backtesting quando passa "step=f" dá o mesmo que executar apenas o "predict"
        _, y_pred = backtesting_forecaster(
            forecaster         = forecaster,
            y                  = df_dados_log[target].y,
            exog               = df_dados_log[atributos_categoricos],
            steps              = f,
            metric             = 'mean_absolute_percentage_error',
            initial_train_size = len(treino),
            refit              = False,
            n_jobs             = NJOBS,
            interval           = [5, 95],
            verbose            = False,
            show_progress      = False
        )

        df_resultado = pd.merge(
            left        = teste.y,
            right       = y_pred,
            left_index  = True,
            right_index = True
        )

        # Precisa ajustar os nomes das colunas para plotar
        df_resultado = df_resultado.rename(columns={
            "pred"        : "{c}".format(c=chave),
            "lower_bound" : "{c}-lo-{n}".format(c=chave, n=intervalos_previsao_plotar[0]),
            "upper_bound" : "{c}-hi-{n}".format(c=chave, n=intervalos_previsao_plotar[0])
        })

        # Antes de plotar, vou retornar da transformação logarítmica
        for c in df_resultado.select_dtypes(include=['float64']).columns.to_list():
            df_resultado[c] = np.expm1(df_resultado[c])

        plot_resultados(
            df_merged  = df_resultado,
            modelo     = "{}".format(chave),
            nome_curto = item[1],
            fh         = f,
            titulo     = "{md} (fh={fh}) ({c})".format(md=chave, fh=f, c=cenario1),
            niveis     = intervalos_previsao_plotar,
            cores      = ["green"],
            pasta_dstn = pasta_resultados+cenario1,
            indice_ds  = True,
            salvar     = SALVAR_PLOTS
        )

# com chuva, com dados categóricos

## StatsForecast

### Baseline - SeasonalNaive

Isso não é um preditor de fato. Ao menos, não se considera assim. Serve como uma baseline a superar.

In [76]:
for f in fh_v:
    # Realiza a separação dos dados neste ponto para cada horizonte de previsão desejado
    treino = df_dados.iloc[:-f, :].copy()
    teste = df_dados.iloc[-f:, :].copy()

    # Exigência da lib MLForecast
    treino = treino.reset_index()
    teste = teste.reset_index()
    treino["unique_id"] = 1
    teste["unique_id"] = 1

    modelo = SeasonalNaive(season_length=365) # Sazonalidade de 365 dias

    stfc = StatsForecast(
        df     = treino,
        models = [modelo],
        freq   = "D",
        n_jobs = NJOBS
    )

    df_futr = teste.drop(columns=target).copy()
    previsoes = stfc.forecast(
        h     = f,
        X_df  = df_futr,
        level = intervalos_previsao
    )

    df_resultado = pd.merge(
        left  = previsoes,
        right = teste[['ds', 'unique_id', 'y']],
        how   = "left",
        on    = ['ds', 'unique_id']
    )

    plot_resultados(
        df_merged  = df_resultado,
        modelo     = modelo.alias,
        nome_curto = "SN",
        fh         = f,
        titulo     = "{md} (fh={fh}) ({c})".format(md=modelo.alias, fh=f, c=cenario2),
        niveis     = intervalos_previsao_plotar,
        cores      = ["green"],
        pasta_dstn = pasta_resultados+cenario2,
        salvar     = SALVAR_PLOTS
    )

## SKForecast

### LinearRegression, LightGBM e CatBoost

A partir da baseline estipulada pelo SeasonalNaive, estes modelos são testados para ver se melhoram os resultados

In [77]:
for f in fh_v:
    treino = df_dados_log.iloc[:-f, :].copy()
    teste = df_dados_log.iloc[-f:, :].copy()

    modelos = {
        "LinearRegression"  : [LinearRegression(), "LR"],
        "CatBoostRegressor" : [CatBoostRegressor(random_seed=SEED, cat_features=atributos_categoricos, verbose=False, allow_writing_files=False), "CB"],
        "LGBMRegressor"     : [LGBMRegressor(random_state=SEED, verbose=-1), "LGBM"]
    }

    # Isso é exclusivo para o LightGBM, passado no momento do 'fit'.
    lgbm_fit_arg = {"categorical_feature": atributos_categoricos}

    for chave, item in modelos.items():
        forecaster = ForecasterAutoreg(
            regressor  = item[0],
            lags       = look_back,
            fit_kwargs = lgbm_fit_arg if chave == "LGBMRegressor" else None
        )

        # Backtesting quando passa "step=f" dá o mesmo que executar apenas o "predict"
        _, y_pred = backtesting_forecaster(
            forecaster         = forecaster,
            y                  = df_dados_log[target].y,
            exog               = df_dados_log.drop(columns=target),
            steps              = f,
            metric             = 'mean_absolute_percentage_error',
            initial_train_size = len(treino),
            refit              = False,
            n_jobs             = NJOBS,
            interval           = [5, 95],
            verbose            = False,
            show_progress      = False
        )

        df_resultado = pd.merge(
            left        = teste.y,
            right       = y_pred,
            left_index  = True,
            right_index = True
        )

        # Precisa ajustar os nomes das colunas para plotar
        df_resultado = df_resultado.rename(columns={
            "pred"        : "{c}".format(c=chave),
            "lower_bound" : "{c}-lo-{n}".format(c=chave, n=intervalos_previsao_plotar[0]),
            "upper_bound" : "{c}-hi-{n}".format(c=chave, n=intervalos_previsao_plotar[0])
        })

        # Antes de plotar, vou retornar da transformação logarítmica
        for col in df_resultado.select_dtypes(include=['float64']).columns.to_list():
            df_resultado[col] = np.expm1(df_resultado[col])

        plot_resultados(
            df_merged  = df_resultado,
            modelo     = "{}".format(chave),
            nome_curto = item[1],
            fh         = f,
            titulo     = "{md} (fh={fh}) ({c})".format(md=chave, fh=f, c=cenario2),
            niveis     = intervalos_previsao_plotar,
            cores      = ["green"],
            pasta_dstn = pasta_resultados+cenario2,
            indice_ds  = True,
            salvar     = SALVAR_PLOTS
        )

## LinearRegression WFV

Walk-forward Validation (WFV) com janela expandida

In [78]:
# fcst = ForecasterAutoreg(
#     regressor = LinearRegression(),
#     lags      = look_back,
# )

# fcst.fit(
#     y    = df_dados_log[target].y,
#     exog = df_dados_log[colunas_chuva+atributos_categoricos],
# )

# data_analise = fcst.get_feature_importances()
# fig = px.histogram(
#     data_frame=data_analise,
#     x=data_analise['importance'],
#     y=data_analise['feature'],
#     barmode='overlay',
#     opacity=1.0,
#     color="feature",
#     orientation='h',
#     color_discrete_sequence=px.colors.qualitative.T10
# )

# fig.update_yaxes(
#     title=dict(
#         text="",
#         font=dict(
#             family="system-ui",
#             size=18
#         )
#     ),
#     zerolinecolor="black",
#     mirror=True,
#     ticks="outside",
#     showline=True,
#     linecolor="black",
# )

# fig.update_xaxes(
#     title=dict(
#         text="Importância",
#         font=dict(
#             family="system-ui",
#             size=18
#         )
#     ),
#     mirror=True,
#     ticks="outside",
#     showline=True,
#     linecolor="black",
# )

# fig.update_layout(
#     width=1500,
#     height=750,
#     plot_bgcolor="#c8d4e3",
#     title=dict(
#         text="Importância de variáveis (LinearRegression)",
#         font=dict(
#             family="system-ui",
#             size=24
#         )
#     ),
# )

# fig.show()

In [79]:
# Este será o resultado a ser superado

f = 1
mdl_lr = LinearRegression()

forecaster = ForecasterAutoreg(
    regressor = mdl_lr,
    lags      = look_back,
)

# Backtesting quando passa "step=f" dá o mesmo que executar apenas o "predict"
_, y_pred = backtesting_forecaster(
    forecaster         = forecaster,
    y                  = df_aux_log[target].y,
    exog               = df_aux_log[atributos_categoricos], #colunas_chuva
    steps              = f,
    metric             = 'mean_absolute_percentage_error',
    initial_train_size = len(df_dados_log),
    refit              = True,
    n_jobs             = NJOBS,
    interval           = [5, 95],
    verbose            = False,
    show_progress      = False
)

df_resultado = pd.merge(
    left        = df_2023_log.y,
    right       = y_pred,
    left_index  = True,
    right_index = True
)

# Precisa ajustar os nomes das colunas para plotar
df_resultado = df_resultado.rename(columns={
    "pred"        : "{}".format("LinearRegression"),
    "lower_bound" : "{c}-lo-{n}".format(c="LinearRegression", n=intervalos_previsao_plotar[0]),
    "upper_bound" : "{c}-hi-{n}".format(c="LinearRegression", n=intervalos_previsao_plotar[0])
})

# Antes de plotar, vou retornar da transformação logarítmica
for c in df_resultado.select_dtypes(include=['float64']).columns.to_list():
    df_resultado[c] = np.expm1(df_resultado[c])

plot_resultados(
    df_merged  = df_resultado,
    modelo     = "{}".format("LinearRegression"),
    nome_curto = "LR_WFV",
    fh         = f,
    titulo     = "{md} (fh={fh}) ({c})".format(md="LinearRegression WFV", fh=f, c=cenario2),
    niveis     = None,
    cores      = ["green"],
    marcadores = False,
    metricas   = "hidrologia",
    pasta_dstn = pasta_resultados+cenario2,
    indice_ds  = True,
    delay      = True,
    salvar     = SALVAR_PLOTS
)

In [80]:
plot_observado_previsao(df_resultado, "LinearRegression", False, pasta_resultados+cenario2)

# Análise de atributos

Cada modelo fará avaliação dos atributos antes de ir para a otimização.

### CatBoost

In [81]:
fc_sel = ForecasterAutoreg(
    lags      = look_back, # Está aqui por obrigação. Mudará quando for pra otimização de parâmetros
    regressor = CatBoostRegressor(random_state=SEED, verbose=False, allow_writing_files=False),
)

sel_RFECV = RFECV(
    estimator              = CatBoostRegressor(random_state=SEED, verbose=False, allow_writing_files=False),
    step                   = 1,
    cv                     = NFOLDS,
    n_jobs                 = NJOBS,
    min_features_to_select = 1, # além das variáveis de chuva, pelo menos mais uma feature; é o mínimo
)

_, selected_exog = select_features(
    forecaster      = fc_sel,
    selector        = sel_RFECV,
    y               = df_dados_log[target].y,
    exog            = df_dados_log.drop(columns=target),
    select_only     = 'exog', # Vai analisar apenas as variáveis exógenas, visto que as lags serão escolhidas na otimização
    force_inclusion = colunas_chuva,
    subsample       = 0.9,
    random_state    = SEED,
    verbose         = False,
)

In [82]:
categoricos_cb = [e for e in atributos_categoricos if e in selected_exog]
categoricos_cb

['dayofyear', 'week']

In [83]:
fcst = ForecasterAutoreg(
    regressor=CatBoostRegressor(random_state=SEED, cat_features=categoricos_cb, verbose=False, allow_writing_files=False),
    lags=look_back, # Está aqui por obrigação. Mudará quando for pra otimização de parâmetros
)

fcst.fit(
    y    = df_dados_log[target].y,
    exog = df_dados_log[colunas_chuva+categoricos_cb],
)

data_analise = fcst.get_feature_importances()
fig = px.histogram(
    data_frame=data_analise,
    x=data_analise['importance'],
    y=data_analise['feature'],
    barmode='overlay',
    opacity=1.0,
    color="feature",
    orientation='h',
    color_discrete_sequence=px.colors.qualitative.T10
)

fig.update_yaxes(
    title=dict(
        text="",
        font=dict(
            family="system-ui",
            size=18
        )
    ),
    zerolinecolor="black",
    mirror=True,
    ticks="outside",
    showline=True,
    linecolor="black",
)

fig.update_xaxes(
    title=dict(
        text="Importância",
        font=dict(
            family="system-ui",
            size=18
        )
    ),
    mirror=True,
    ticks="outside",
    showline=True,
    linecolor="black",
)

fig.update_layout(
    width=1500,
    height=750,
    plot_bgcolor="#c8d4e3",
    title=dict(
        text="Importância de variáveis (CatBoost)",
        font=dict(
            family="system-ui",
            size=24
        )
    ),
)

fig.show()

### LightGBM

In [84]:
fc_sel = ForecasterAutoreg(
    lags      = look_back, # Está aqui por obrigação. Mudará quando for pra otimização de parâmetros
    regressor = LGBMRegressor(random_state=SEED, verbose=-1)
)

sel_RFECV = RFECV(
    step                   = 1,
    cv                     = NFOLDS,
    n_jobs                 = NJOBS,
    estimator              = LGBMRegressor(random_state=SEED, verbose=-1),
    min_features_to_select = 1, # além das variáveis de chuva, pelo menos mais uma feature; é o mínimo
)

_, selected_exog = select_features(
    forecaster      = fc_sel,
    selector        = sel_RFECV,
    y               = df_dados_log[target].y,
    exog            = df_dados_log.drop(columns=target),
    select_only     = 'exog', # Vai analisar apenas as variáveis exógenas, visto que as lags serão escolhidas na otimização
    force_inclusion = colunas_chuva,
    subsample       = 0.9,
    random_state    = SEED,
    verbose         = False,
)

In [85]:
categoricos_lgbm = [e for e in atributos_categoricos if e in selected_exog]
categoricos_lgbm

['dayofyear', 'week']

In [86]:
lgbm_fit_arg = {"categorical_feature": categoricos_lgbm}

fcst = ForecasterAutoreg(
    lags       = look_back, # Está aqui por obrigação. Mudará quando for pra otimização de parâmetros
    regressor  = LGBMRegressor(random_state=SEED, verbose=-1),
    fit_kwargs = lgbm_fit_arg
)

fcst.fit(
    y    = df_dados_log[target].y,
    exog = df_dados_log[colunas_chuva+categoricos_lgbm],
)

data_analise = fcst.get_feature_importances()
fig = px.histogram(
    data_frame=data_analise,
    x=data_analise['importance'],
    y=data_analise['feature'],
    barmode='overlay',
    opacity=1.0,
    color="feature",
    orientation='h',
    color_discrete_sequence=px.colors.qualitative.T10
)

fig.update_yaxes(
    title=dict(
        text="",
        font=dict(
            family="system-ui",
            size=18
        )
    ),
    zerolinecolor="black",
    mirror=True,
    ticks="outside",
    showline=True,
    linecolor="black",
)

fig.update_xaxes(
    title=dict(
        text="Importância",
        font=dict(
            family="system-ui",
            size=18
        )
    ),
    mirror=True,
    ticks="outside",
    showline=True,
    linecolor="black",
)

fig.update_layout(
    width=1500,
    height=750,
    plot_bgcolor="#c8d4e3",
    title=dict(
        text="Importância de variáveis (LGBMRegressor)",
        font=dict(
            family="system-ui",
            size=24
        )
    ),
)
fig.show()

# Otimização de hiperparâmetros

### CatBoost

In [87]:
# def cb_search_space(trial):
#     search_space  = {
#         # Parâmetros a otimizar
#         'learning_rate'     : trial.suggest_float('learning_rate', 1e-5, 5e-1),
#         'colsample_bylevel' : trial.suggest_float('colsample_bylevel', 0.05, 1.0),
#         'subsample'         : trial.suggest_float('subsample', 0.05, 1.0),
#         'min_data_in_leaf'  : trial.suggest_int("min_data_in_leaf", 1, 100),

#         # Parâmetro para o Forecaster
#         'lags'             : trial.suggest_int('lags', 1, 21),
#     }
#     return search_space

# f = 1 # fh = fh_v[0]

# mdl_cb = CatBoostRegressor(
#     # Parâmetros fixos
#     random_seed         = SEED,
#     cat_features        = categoricos_cb,
#     verbose             = False,
#     allow_writing_files = False
# )

# fc_bs = ForecasterAutoreg(
#     regressor = mdl_cb,
#     lags      = look_back, # obrigatório estar aqui, mas é alterado pelo otimizador
# )

# _, best_trial = bayesian_search_forecaster(
#     y                     = df_dados_log.y,
#     exog                  = df_dados_log[colunas_chuva+categoricos_cb].convert_dtypes(), # Macete bizarro pra parar de quebrar o CatBoost
#     forecaster            = fc_bs,
#     search_space          = cb_search_space,
#     steps                 = f,
#     refit                 = True,
#     n_trials              = 50,
#     metric                = 'mean_absolute_percentage_error',
#     initial_train_size    = len(df_dados_log.iloc[:(-NFOLDS*f), :]),
#     fixed_train_size      = False,
#     return_best           = True,
#     n_jobs                = NJOBS,
#     random_state          = SEED,
#     verbose               = False,
#     show_progress         = True,
#     engine                = 'optuna',
#     output_file           = pasta_resultados+cenario2+"/opt/cb_opt_fh1.txt",
#     kwargs_create_study   = {'study_name' : 'cb_opt_fh1', 'direction'  : 'minimize'},
#     kwargs_study_optimize = {'catch' : (FloatingPointError, ValueError, RuntimeError)},
# )

  0%|          | 0/50 [00:00<?, ?it/s]

[I 2024-08-17 19:51:10,494] Trial 0 finished with value: 0.01522368315919605 and parameters: {'learning_rate': 0.00438777719072525, 'colsample_bylevel': 0.33380828975267884, 'subsample': 0.10359337014100449, 'min_data_in_leaf': 49, 'lags': 6}. Best is trial 0 with value: 0.01522368315919605.
[I 2024-08-17 19:51:23,496] Trial 1 finished with value: 0.012689979389184586 and parameters: {'learning_rate': 0.39713905888148177, 'colsample_bylevel': 0.32721719106674185, 'subsample': 0.748020571461332, 'min_data_in_leaf': 23, 'lags': 16}. Best is trial 1 with value: 0.012689979389184586.
[I 2024-08-17 19:51:38,065] Trial 2 finished with value: 0.011608771286233019 and parameters: {'learning_rate': 0.025876865585929585, 'colsample_bylevel': 0.520741923071318, 'subsample': 0.6793751937428989, 'min_data_in_leaf': 75, 'lags': 15}. Best is trial 2 with value: 0.011608771286233019.
[I 2024-08-17 19:51:50,155] Trial 3 finished with value: 0.013783633703578884 and parameters: {'learning_rate': 0.12718

##### Walk-forward Validation com janela expandida

Uma vez com os parâmetros ótimos em mãos, executa o que é considerado o padrão ouro na avaliação de modelos de previsão: executa um fit passo-a-passo num longo período de avaliação.

No caso em questão, irei executar um WFV ao longo do ano de 2023 inteiro, fazendo "fit" dia-a-dia e prevendo um dia por vez.

In [88]:
f = 1
mdl_cb_opt = CatBoostRegressor(
    # Parâmetros fixos
    random_seed         = SEED,
    cat_features        = categoricos_cb,
    verbose             = False,
    allow_writing_files = False,

    # Parâmetros ótimos encontrados
    learning_rate     = round(0.19313578097991652, 5),
    colsample_bylevel = round(0.6774778026468194,  5),
    subsample         = round(0.5648608138473864,  5),
    min_data_in_leaf  = 36,
)

forecaster = ForecasterAutoreg(
    regressor = mdl_cb_opt,
    lags      = 9,
)

# Backtesting quando passa "step=f" dá o mesmo que executar apenas o "predict"
_, y_pred = backtesting_forecaster(
    forecaster         = forecaster,
    y                  = df_aux_log[target].y,
    exog               = df_aux_log[colunas_chuva+categoricos_cb].convert_dtypes(),
    steps              = f,
    metric             = 'mean_absolute_percentage_error',
    initial_train_size = len(df_dados_log),
    refit              = True,
    n_jobs             = NJOBS,
    interval           = [5, 95],
    verbose            = False,
    show_progress      = False
)

df_resultado = pd.merge(
    left        = df_2023_log.y,
    right       = y_pred,
    left_index  = True,
    right_index = True
)

# Precisa ajustar os nomes das colunas para plotar
df_resultado = df_resultado.rename(columns={
    "pred"        : "{}".format("CatBoostRegressor"),
    "lower_bound" : "{c}-lo-{n}".format(c="CatBoostRegressor", n=intervalos_previsao_plotar[0]),
    "upper_bound" : "{c}-hi-{n}".format(c="CatBoostRegressor", n=intervalos_previsao_plotar[0])
})

# Antes de plotar, vou retornar da transformação logarítmica
for col in df_resultado.select_dtypes(include=['float64']).columns.to_list():
    df_resultado[col] = np.expm1(df_resultado[col])

plot_resultados(
    df_merged  = df_resultado,
    modelo     = "{}".format("CatBoostRegressor"),
    nome_curto = "CB_WFV",
    fh         = f,
    titulo     = "{md} (fh={fh}) ({c})".format(md="CatBoostRegressor WFV", fh=f, c=cenario2),
    niveis     = None,
    cores      = ["green"],
    metricas   = "hidrologia",
    marcadores = False,
    pasta_dstn = pasta_resultados+cenario2,
    indice_ds  = True,
    delay      = True,
    salvar     = SALVAR_PLOTS
)

/home/wasf84/bin/miniconda3/envs/py39/lib/python3.9/site-packages/skforecast/model_selection/model_selection.py:475: LongTrainingWarning:

The forecaster will be fit 365 times. This can take substantial amounts of time. If not feasible, try with `refit = False`.
 
 You can suppress this warning using: warnings.simplefilter('ignore', category=LongTrainingWarning)

/home/wasf84/bin/miniconda3/envs/py39/lib/python3.9/site-packages/sktime/performance_metrics/forecasting/_classes.py:615: UserWarning:

y_pred and y_true do not have the same column index. This may indicate incorrect objects passed to the metric. Indices of y_true will be used for y_pred.



In [89]:
plot_observado_previsao(df_resultado, "CatBoostRegressor", False, pasta_resultados+cenario2)

### LightGBM

In [90]:
# def lgbm_search_space(trial):
#     search_space  = {
#         # Parâmetros a otimiziar
#         'learning_rate'    : trial.suggest_float('learning_rate', 1e-5, 5e-1),
#         'colsample_bytree' : trial.suggest_float('colsample_bytree', 0.05, 1.0),
#         'subsample'        : trial.suggest_float('subsample', 0.05, 1.0),
#         'min_data_in_leaf' : trial.suggest_int("min_data_in_leaf", 1, 100),

#         # Parâmetro para o Forecaster
#         'lags'             : trial.suggest_int('lags', 1, 21),
#     }
#     return search_space

# f = 1 # fh = fh_v[0]
# lgbm_fit_arg = {"categorical_feature": categoricos_lgbm}

# mdl_lgbm = LGBMRegressor(
#     # Parâmetros fixos
#     random_seed  = SEED,
#     bagging_freq = 1,
#     verbose      = -1,
# )

# fc_bs = ForecasterAutoreg(
#     regressor  = mdl_lgbm,
#     lags       = look_back, # obrigatório estar aqui, mas é alterado pelo otimizador
#     fit_kwargs = lgbm_fit_arg
# )

# _, best_trial = bayesian_search_forecaster(
#     y                     = df_dados_log.y,
#     exog                  = df_dados_log[colunas_chuva+categoricos_lgbm],
#     forecaster            = fc_bs,
#     search_space          = lgbm_search_space,
#     steps                 = f,
#     refit                 = True,
#     n_trials              = 50,
#     metric                = 'mean_absolute_percentage_error',
#     initial_train_size    = len(df_dados_log.iloc[:(-NFOLDS*f), :]),
#     fixed_train_size      = False,
#     return_best           = True,
#     n_jobs                = NJOBS,
#     random_state          = SEED,
#     verbose               = False,
#     show_progress         = True,
#     engine                = 'optuna',
#     output_file           = pasta_resultados+cenario2+"/opt/lgbm_opt_fh1.txt",
#     kwargs_create_study   = {'study_name' : 'lgbm_opt_fh1', 'direction'  : 'minimize'},
#     kwargs_study_optimize = {'catch' : (FloatingPointError, ValueError, RuntimeError)},
# )

  0%|          | 0/50 [00:00<?, ?it/s]

[I 2024-08-17 20:08:41,041] Trial 0 finished with value: 0.01819707939137183 and parameters: {'learning_rate': 0.00438777719072525, 'colsample_bytree': 0.33380828975267884, 'subsample': 0.10359337014100449, 'min_data_in_leaf': 49, 'lags': 6}. Best is trial 0 with value: 0.01819707939137183.
[I 2024-08-17 20:08:41,407] Trial 1 finished with value: 0.014208398359591335 and parameters: {'learning_rate': 0.39713905888148177, 'colsample_bytree': 0.32721719106674185, 'subsample': 0.748020571461332, 'min_data_in_leaf': 23, 'lags': 16}. Best is trial 1 with value: 0.014208398359591335.
[I 2024-08-17 20:08:41,666] Trial 2 finished with value: 0.013242334417593229 and parameters: {'learning_rate': 0.025876865585929585, 'colsample_bytree': 0.520741923071318, 'subsample': 0.6793751937428989, 'min_data_in_leaf': 75, 'lags': 15}. Best is trial 2 with value: 0.013242334417593229.
[I 2024-08-17 20:08:41,911] Trial 3 finished with value: 0.018580712803166484 and parameters: {'learning_rate': 0.12718730

##### Walk-forward Validation com janela expandida

Uma vez com os parâmetros ótimos em mãos, executa o que é considerado o padrão ouro na avaliação de modelos de previsão: executa um fit passo-a-passo num longo período de avaliação.

No caso em questão, irei executar um WFV ao longo do ano de 2023 inteiro, fazendo "fit" dia-a-dia e prevendo um dia por vez.

In [91]:
f = 1
lgbm_fit_arg = {"categorical_feature": categoricos_lgbm}

mdl_lgbm = LGBMRegressor(
    # Parâmetros fixos
    random_seed  = SEED,
    bagging_freq = 1,
    verbose      = -1,

    # Parâmetros ótimos encontrados
    learning_rate    = round(0.16313456320632674, 5),
    colsample_bytree = round(0.8778417060012838,  5),
    subsample        = round(0.4310639368475818,  5),
    min_data_in_leaf = 66,
)

fc_lgbm_opt = ForecasterAutoreg(
    regressor  = mdl_lgbm,
    lags       = 21,
    fit_kwargs = lgbm_fit_arg,
)

_, y_pred = backtesting_forecaster(
    forecaster         = fc_lgbm_opt,
    y                  = df_aux_log[target].y,
    exog               = df_aux_log[colunas_chuva+categoricos_lgbm],
    steps              = f,
    metric             = 'mean_absolute_percentage_error',
    initial_train_size = len(df_dados_log),
    refit              = True,
    n_jobs             = NJOBS,
    interval           = [5, 95],
    verbose            = False,
    show_progress      = False
)

df_resultado = pd.merge(
    left        = df_2023_log.y,
    right       = y_pred,
    left_index  = True,
    right_index = True
)

# Precisa ajustar os nomes das colunas para plotar
df_resultado = df_resultado.rename(columns={
    "pred"        : "{}".format("LGBMRegressor"),
    "lower_bound" : "{c}-lo-{n}".format(c="LGBMRegressor", n=intervalos_previsao_plotar[0]),
    "upper_bound" : "{c}-hi-{n}".format(c="LGBMRegressor", n=intervalos_previsao_plotar[0])
})

# Antes de plotar, vou retornar da transformação logarítmica
for col in df_resultado.select_dtypes(include=['float64']).columns.to_list():
    df_resultado[col] = np.expm1(df_resultado[col])

plot_resultados(
    df_merged  = df_resultado,
    modelo     = "{}".format("LGBMRegressor"),
    nome_curto = "LGBM_WFV",
    fh         = f,
    titulo     = "{md} (fh={fh}) ({c})".format(md="LGBMRegressor WFV", fh=f, c=cenario2),
    niveis     = None,
    cores      = ["green"],
    marcadores = False,
    metricas   = "hidrologia",
    pasta_dstn = pasta_resultados+cenario2,
    indice_ds  = True,
    delay      = True,
    salvar     = SALVAR_PLOTS
)

/home/wasf84/bin/miniconda3/envs/py39/lib/python3.9/site-packages/skforecast/model_selection/model_selection.py:475: LongTrainingWarning:

The forecaster will be fit 365 times. This can take substantial amounts of time. If not feasible, try with `refit = False`.
 
 You can suppress this warning using: warnings.simplefilter('ignore', category=LongTrainingWarning)

/home/wasf84/bin/miniconda3/envs/py39/lib/python3.9/site-packages/sktime/performance_metrics/forecasting/_classes.py:615: UserWarning:

y_pred and y_true do not have the same column index. This may indicate incorrect objects passed to the metric. Indices of y_true will be used for y_pred.



In [92]:
plot_observado_previsao(df_resultado, "LGBMRegressor", False, pasta_resultados+cenario2)

# FIM